## *Jeu de données IRVE*
-------------------
# Analyse des relations entre champs (intégrité des données)
------------------------------
## Contexte
- clarification du rôle des modèles de données dans les jeux de données (cf mise à jour récente des [guides data.gouv](https://guides.etalab.gouv.fr/qualite/documenter-les-donnees/))
- intégration d'une propriété "relationship" dans les schémas de données ([issue TableSchema](https://github.com/frictionlessdata/specs/issues/803) en cours de validation)
- création d'outils de contrôle des relations entre champs des jeux de données tabulaires (cf usage ci-dessous)

## Objectifs
- valider sur un cas réel l'utilisation d'un modèle de données en complément d'un schéma de données
- identifier les apports que pourraient avoir les contrôles de validation des relations entre champs

## Résultats
- les principaux résultats sont regroupés sur un [tableau de bord](https://nbviewer.org/github/loco-philippe/Environmental-Sensing/blob/main/python/Validation/irve/Analyse/IRVE_indicateurs.ipynb) 
- la formalisation d'un modèle de données facilite la compréhension des données et des relations entre champs
- l'outil de contrôle permet d'améliorer significativement la qualité des données par l'identification d'incohérences de relations
- l'identification des incohérences permet de trouver des stratégies de réduction des écarts (dans l'exemple ci-dessous, on passe 36% d'écart à 3,8 %)  
- l'analyse des données permet de (re)construire le modèle de données qui minimise les incohérences
- les incohérences détectées sur le jeu de données IRVE restent faibles (inférieures à 5 % des point de charge documentés - voir chapitre 4)

## Suite à donner
- Mettre à jour, valider et publier le modèle de données IRVE
- Définir les contrôles supplémentaires à intégrer pour toutes nouvelles données ainsi que pour le jeu complet
- Mettre en oeuvre les outils de contrôle

## Evolutions possibles 
- Ajouter dans les guides d'Etalab un guide pour les modèles de données 
- Intégrer dans les schémas de données la propriété "relationship" en cours de validation,
- Définir un indicateur qui mesure l'écart (existant / attendu) des relations entre champs

## Sommaire
*(liens actifs sur jupyter Notebook ou Nbviewer)*
- [1 - modèle de données](#1---modèle-de-données)
- [2 - Initialisation](#2---Initialisation)
- [3 - Séparation des pdc itinerance et hors itinerance](#3---Séparation-des-pdc-itinerance-et-hors-itinerance)
- [4 - Bilan initial intégrité](#4---Bilan-initial-intégrité)
- [5 - Séparation doublons pdc - date de maj](#5---Séparation-doublons-pdc---date-de-maj)
- [6 - Séparation doublons station - date de maj](#6---Séparation-doublons-station---date-de-maj)
- [7 - Synthèse](#7---Synthèse)
- [8 - Exemples d'erreurs résiduelles](#8---Exemples-d\'erreurs-résiduelles)

Ce Notebook peut être consulté sur [nbviewer](http://nbviewer.org/github/loco-philippe/Environmental-Sensing/tree/main/python/Validation/irve/Analyse)

données utilisées : https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/    
fichier : "*consolidation-etalab-schema-irve-statique-v-2.2.0-2024xxxx.csv*"

-----------------------
# 1 - modèle de données

Le modèle de données proposé ci-dessous est construit sur la base du schéma de données mis à disposition et du contenu du jeu de données.    
Il est à consolider en fonction de l'expertise des concepteurs et réutilisateurs (voir [guide méthodologique](https://github.com/loco-philippe/Environmental-Sensing/blob/main/property_relationship/FR_methodology.ipynb)).

*Notation:*
- *M : Mandatory - documentation obligatoire*
- *PK : Primary Key - identifiant unique de l'entité*
- *Root : champ fictif associé à une ligne du tableau*

In [1]:
from base64 import b64encode
from IPython.display import Image, display
with open('IRVE_modele.txt', 'r', encoding="utf-8") as f:
    modele = f.read()
display(Image(url="https://mermaid.ink/img/" + b64encode(modele.encode("ascii")).decode("ascii")))

------
# 2 - Initialisation
## initialisation logicielle

In [2]:
from datetime import datetime
import json
from tab_dataset import Cdataset
import pandas as pd
import ntv_pandas as npd
import pathlib

def analyse_integrite(data, schema, affiche=True, indic=True):
    '''analyse les relations du DataFrame 'data' définies dans le schéma 'schema'.
    Le nombre de ligne en erreur par relation (dict) est retourné et optionnellement affiché (paramètre 'affiche=True') . 
    Les lignes en erreur sont optionnellement ajoutées (paramètre 'indic=True') à 'data' sous forme de champs booléens par relation.
    '''
    dic_errors = Cdataset(data).check_relationship(schema)
    dic_count = {name: len(errors) for name, errors in dic_errors.items()}
    if affiche:
        for name, total in dic_count.items():
            print('{:<50} {:>5}'.format(name, total))
    if indic:
        data['ok'] = True
        for name, errors in dic_errors.items():
            data[name] = True
            data.loc[errors, name] = False
            data['ok'] = data['ok'] & data[name] 
    return dic_count

def add_nbre_pdc(data, affiche=True):
    '''ajoute un champ avec le nombre de pdc calculés et le compare au champ 'nbre_pdc'
    Le nombre d'écarts est affiché optionnellement (paramètre 'affiche=True').
    '''
    data['nb_pdc_calc'] = data.groupby('id_station_itinerance')['index'].transform('count')
    nbre_pdc = data['nbre_pdc'].astype('int')
    data['nbre_pdc_ok'] = data['nb_pdc_calc'] == nbre_pdc
    data['calc-nbre'] = data['nb_pdc_calc'] - nbre_pdc
    if affiche:
        ecart_nbre = len(data) - data['nbre_pdc_ok'].sum()
        print('{:<20} {:>5}'.format('nbre_pdc_ko', ecart_nbre))
        print(round(ecart_nbre / len(data) * 100), ' %')

## initialisation des données
- lecture du fichier issu de l'api

In [3]:
dates = ['2023-03-03', '2023-04-17', '2023-05-02', '2023-05-24', '2023-06-17', '2023-07-04', '2023-07-23', '2023-08-08',
         '2023-09-06', '2023-09-19', '2023-10-04', '2023-10-21', '2023-10-31', '2023-11-23', '2023-12-14', '2023-12-29',
         '2024-01-15', '2024-02-07', '2024-02-28', '2024-03-27', '2024-04-24', '2024-05-19', '2024-06-05', '2024-08-02',
         '2024-10-24', '2024-11-27', '2025-01-03']
date= dates[-1]
log = {'date_irve': date, 
    'file': 'consolidation-etalab-schema-irve-statique-v-2.3.1-'+date[:4]+date[5:7]+date[8:]+'.csv', # après 2024-02-06
    'chemin': str(pathlib.Path(npd.__file__).parent.parent.parent/"Environmental-Sensing"/"python"/"Validation"/"irve"/"Analyse")
      }
irve = pd.read_csv(log['chemin'] + '/' + log['file'], sep=',', low_memory=False, dtype='object')
log['len_irve'] = len(irve)
print('nombre de lignes : ', log['len_irve']) 

nombre de lignes :  119394


## schéma de données
Le schéma de données restreint à la propriété 'relationship' et construit à partir du modèle de données est le suivants :

In [4]:
# complément à inclure dans le schéma de données
schema = {
    'relationships': [
         # relation unicité des pdl
         {"fields": ["id_pdc_itinerance", "index"],                    "link" : "coupled" },   
         # relations inter entités
         {"fields": ["id_station_itinerance", "contact_operateur"],    "link" : "derived" },
         {"fields": ["id_station_itinerance", "nom_enseigne"],         "link" : "derived" },
         {"fields": ["id_station_itinerance", "coordonneesXY"],        "link" : "derived" },
         {"fields": ["id_pdc_itinerance", "id_station_itinerance"],    "link" : "derived" },
         # relations intra entité - station
         {"fields": ["id_station_itinerance", "nom_station"],          "link" : "derived" },
         {"fields": ["id_station_itinerance", "implantation_station"], "link" : "derived" },
         #{"fields": ["id_station_itinerance", "date_maj"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "nbre_pdc"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "condition_acces"],      "link" : "derived" },
         {"fields": ["id_station_itinerance", "horaires"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "station_deux_roues"],   "link" : "derived" },
         # relations intra entité - localisation
         {"fields": ["coordonneesXY", "adresse_station"],              "link" : "derived" }
    ]
}
# liste des champs liés à un controle (relations) et obligatoires (mandatory)
relations = ['index', 'contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 
             'nom_station', 'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 
             'id_pdc_itinerance', 'date_maj', 'last_modified']
mandatory = ['contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 'nom_station',
             'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 'id_pdc_itinerance', 
             'puissance_nominale', 'prise_type_ef', 'prise_type_2', 'prise_type_combo_ccs', 'prise_type_chademo', 
             'prise_type_autre', 'paiement_acte', 'paiement_autre', 'reservation',  'accessibilite_pmr', 'restriction_gabarit', 
             'date_maj', 'last_modified']
controles = ['index - id_pdc_itinerance', 'contact_operateur - id_station_itinerance', 'nom_enseigne - id_station_itinerance',
             'coordonneesXY - id_station_itinerance', 'id_station_itinerance - id_pdc_itinerance',
             'nom_station - id_station_itinerance', 'implantation_station - id_station_itinerance',
             'nbre_pdc - id_station_itinerance', 'condition_acces - id_station_itinerance', 'horaires - id_station_itinerance',
             'station_deux_roues - id_station_itinerance', 'adresse_station - coordonneesXY']

-----------------------
## 3 - Séparation des pdc itinerance et hors itinerance
- de l'ordre de 3 % des points de charge sont hors itinerance

In [5]:
# séparation des données
data = irve
data[['id_station_itinerance','id_pdc_itinerance','restriction_gabarit']] = data[['id_station_itinerance','id_pdc_itinerance','restriction_gabarit']].astype('string')
#data[['last_modified','date_maj']] = data[['last_modified','date_maj']].astype('datetime64')
data['non_concerne'] = data['id_station_itinerance'].str.contains('oncern') | data['id_pdc_itinerance'].str.contains('oncern')

non_concerne = data[data['non_concerne']].reset_index()['index']
itinerance = data[~data['non_concerne']].reset_index()
itinerance_init = itinerance.loc[:, relations]
log['pdc_hors_itinerance'] = len(non_concerne)
log['pdc_en_itinerance'] = len(itinerance)
print('nombre de pdc hors itinerance : ', log['pdc_hors_itinerance'], 'soit : ', len(non_concerne)/(len(non_concerne)+len(itinerance)))
print('nombre de pdc en itinerance   : ', log['pdc_en_itinerance'])

nombre de pdc hors itinerance :  1135 soit :  0.009506340352111496
nombre de pdc en itinerance   :  118259


In [6]:
# arborescence des champs et nombre de valeurs différentes
print(itinerance_init.npd.analysis().tree())

-1: root-derived (118259)
   0 : index (0 - 118259)
   1 : contact_operateur (118068 - 191)
   2 : nom_enseigne (114554 - 3705)
   3 : coordonneesXY (87460 - 30799)
   4 : adresse_station (87840 - 30419)
   5 : id_station_itineranc (68942 - 49317)
   6 : nom_station (86668 - 31591)
   7 : implantation_station (118254 - 5)
   8 : nbre_pdc (118191 - 68)
   9 : condition_acces (118257 - 2)
   10: horaires (117372 - 887)
   11: station_deux_roues (118253 - 6)
   12: id_pdc_itinerance (9257 - 109002)
   13: date_maj (117402 - 857)
   14: last_modified (117665 - 594)


-----------------------
## 4 - Bilan initial intégrité
- 19 % des lignes présentent un défaut d'intégrité

In [7]:
# séparation données bonnes (itinerance_ok_1) et données résiduelles (itinerance_1)
res = analyse_integrite(itinerance_init, schema)
itinerance_ok_1 = itinerance_init.loc[itinerance_init.ok, relations].reset_index(drop=True)
itinerance_1 = itinerance_init.loc[~itinerance_init.ok, relations].reset_index(drop=True)
itinerance_init = itinerance_init.loc[:, relations]
log['init_ok'] = len(itinerance_ok_1)
log['init_ko'] = len(itinerance_1)
print("\nnombre d'enregistrements sans erreurs : ", log['init_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['init_ko'])
print("taux d'erreur : ", round(log['init_ko'] / log['pdc_en_itinerance'] * 100), ' %')

index - id_pdc_itinerance                          18512
contact_operateur - id_station_itinerance           7556
nom_enseigne - id_station_itinerance                8199
coordonneesXY - id_station_itinerance               9003
id_station_itinerance - id_pdc_itinerance          10422
nom_station - id_station_itinerance                 8233
implantation_station - id_station_itinerance        3751
nbre_pdc - id_station_itinerance                    3101
condition_acces - id_station_itinerance               83
horaires - id_station_itinerance                      60
station_deux_roues - id_station_itinerance          7979
adresse_station - coordonneesXY                     2736

nombre d'enregistrements sans erreurs :  95480
nombre d'enregistrements avec au moins une erreur :  22779
taux d'erreur :  19  %


-----------------------
## 5 - Séparation doublons pdc - date de maj
- 35% des pdc en erreur sont liées aux doublons de pdc
- la suppression des doublons permet de réduire de 50% le nombre de lignes erronnées

In [8]:
# séparation doublons pdc (doublons_pdc) et données résiduelles (itinerance_2)
itinerance_1['doublons_pdc'] = itinerance_1.sort_values(by=['date_maj', 'last_modified']).duplicated('id_pdc_itinerance', keep='last')

doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)['index']
#doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_2 = itinerance_1[~itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_1 = itinerance_1.loc[:, relations]
log['doublons_pdc'] = len(doublons_pdc)
log['sans_doublons_pdc'] = len(itinerance_2)
print('nombre de doublons pdc : ', log['doublons_pdc'], ' soit ', round(log['doublons_pdc']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_pdc'], ' soit ', round(log['sans_doublons_pdc']/log['init_ko'] * 100), ' %')

nombre de doublons pdc :  9257  soit  41  %
nombre de pdc sans doublon   :  13522  soit  59  %


In [9]:
# séparation données bonnes (itinerance_ok_3) et données résiduelles (itinerance_3)
res = analyse_integrite(itinerance_2, schema)
itinerance_ok_3 = itinerance_2.loc[itinerance_2.ok, relations].reset_index(drop=True)
itinerance_3 = itinerance_2.loc[~itinerance_2.ok, relations].reset_index(drop=True)
itinerance_2 = itinerance_2.loc[:, relations]
log['etape3_ok'] = len(itinerance_ok_3)
log['etape3_ko'] = len(itinerance_3)
print("\nnombre d'enregistrements sans erreurs : ", log['etape3_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape3_ko'])

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance             80
nom_enseigne - id_station_itinerance                  91
coordonneesXY - id_station_itinerance                850
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  722
implantation_station - id_station_itinerance         226
nbre_pdc - id_station_itinerance                     735
condition_acces - id_station_itinerance               20
horaires - id_station_itinerance                      37
station_deux_roues - id_station_itinerance           504
adresse_station - coordonneesXY                     2567

nombre d'enregistrements sans erreurs :  9173
nombre d'enregistrements avec au moins une erreur :  4349


-----------------------
## 6 - Séparation doublons station - date de maj
- la suppression des anciens pdc permet de réduire de 25% supplémentaire le nombre de lignes erronnées
- les dernières erreurs correspondent à des stations associées à 43 opérateurs et sont liées à des causes multiples

In [10]:
# séparation doublons stations (doublons_stat_maj) et données résiduelles (itinerance_4)
itinerance_3['stat_maj'] = itinerance_3.id_station_itinerance + itinerance_3.date_maj
stat_maj_unique = itinerance_3.sort_values(by='stat_maj').drop_duplicates('id_station_itinerance', keep='last')
itinerance_3['last_stat_maj'] = itinerance_3['stat_maj'].isin(stat_maj_unique['stat_maj'])

doublons_stat_maj = itinerance_3[~itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)['index']
itinerance_4 = itinerance_3[itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)
itinerance_3 = itinerance_3.loc[:, relations]
log['doublons_station'] = len(doublons_stat_maj)
log['sans_doublons_station'] = len(itinerance_4)
print('nombre de doublons stations : ', log['doublons_station'], ' soit ', 
      round(log['doublons_station']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_station'])

nombre de doublons stations :  115  soit  1  %
nombre de pdc sans doublon   :  4234


In [11]:
# séparation données bonnes (itinerance_ok_5) et données résiduelles (itinerance_5 / itinerance_5_full)
res = analyse_integrite(itinerance_4, schema)
itinerance_ok_5 = itinerance_4.loc[itinerance_4.ok, relations].reset_index(drop=True)
itinerance_5_full = itinerance_4.loc[~itinerance_4.ok].reset_index(drop=True)
itinerance_5 = itinerance_5_full.loc[:, relations]
#itinerance_4 = itinerance_4.loc[:, relations]
log['etape5_ok'] = len(itinerance_ok_5)
log['etape5_ko'] = len(itinerance_5)
print("\nnombre d'enregistrements sans erreurs : ", log['etape5_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape5_ko'], ' soit ', 
      round(log['etape5_ko']/log['etape3_ko'] * 100), ' %')

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              2
nom_enseigne - id_station_itinerance                  13
coordonneesXY - id_station_itinerance                666
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  614
implantation_station - id_station_itinerance         194
nbre_pdc - id_station_itinerance                     567
condition_acces - id_station_itinerance                0
horaires - id_station_itinerance                      34
station_deux_roues - id_station_itinerance           448
adresse_station - coordonneesXY                     2519

nombre d'enregistrements sans erreurs :  108
nombre d'enregistrements avec au moins une erreur :  4126  soit  95  %


In [12]:
# structure des données présentant une erreur et nombre de valeurs
print(itinerance_5.npd.analysis().tree())

-1: root-derived (4126)
   0 : index (0 - 4126)
   1 : contact_operateur (4064 - 62)
   2 : nom_enseigne (3932 - 194)
      9 : condition_acces (192 - 2)
   3 : coordonneesXY (3356 - 770)
   4 : adresse_station (2941 - 1185)
   5 : id_station_itineranc (2331 - 1795)
      13: date_maj (1656 - 139)
   6 : nom_station (3001 - 1125)
   7 : implantation_station (4121 - 5)
   8 : nbre_pdc (4092 - 34)
   10: horaires (4062 - 64)
   11: station_deux_roues (4120 - 6)
   12: id_pdc_itinerance (0 - 4126)
   14: last_modified (4015 - 111)


In [13]:
# nb maxi d'erreurs
itinerance_4['somme'] = 0
for name in res.keys():
    itinerance_4['somme'] += 1 - itinerance_4[name]
erreurs = max(itinerance_4['somme'])
maxi = itinerance_4[itinerance_4.somme >= erreurs]
print("nombre d'enregistrements avec ", erreurs, " erreurs : ", len(maxi))

nombre d'enregistrements avec  4  erreurs :  4


In [71]:
# principaux opérateurs contributeur de défauts
operateurs = list(itinerance_5.drop_duplicates('contact_operateur')['contact_operateur'])
erreurs_op = [len(itinerance_5.loc[itinerance_5.contact_operateur == operateur]) for operateur in operateurs]
err_op_5, oper_5 = tuple(zip(*(sorted(zip(erreurs_op, operateurs), reverse=True))[:5]))
err_op_10, oper_10 = tuple(zip(*(sorted(zip(erreurs_op, operateurs), reverse=True))[:10]))

print(len(err_op_5), ' opérateurs représentent : ', round(sum(err_op_5) / sum(erreurs_op) * 100), ' % des écarts\n')
for err, op in zip(err_op_5, oper_5):
    print(err, op)

5  opérateurs représentent :  62  % des écarts

799 support@alizecharge.fr
627 supervision-ev.france@totalenergies.com
582 sav@izivia.com
326 a.buffler@bouygues-es.com
231 cpo@citeos.com


In [72]:
print(len(err_op_10), ' opérateurs représentent : ', round(sum(err_op_10) / sum(erreurs_op) * 100), ' % des écarts\n')
for err, op in zip(err_op_10, oper_10):
    print(err, op)

10  opérateurs représentent :  80  % des écarts

799 support@alizecharge.fr
627 supervision-ev.france@totalenergies.com
582 sav@izivia.com
326 a.buffler@bouygues-es.com
231 cpo@citeos.com
217 info@ionity.eu
142 exploitation@r3-charge.fr
131 contact@e-totem.fr
127 hello@powerdot.fr
110 customerservice@shellrecharge.com


-----------------------
## 7 - Synthèse


### fichiers
Génération des fichiers intégrant les défauts d'intégrité :
- fichier csv des lignes résiduelles à traiter (IRVE_itinerance_residuel)
- fichier csv des données itinerance avec indicateur des données à corriger ou à ignorer (IRVE_itinerance_complet)
- fichier csv des données itinerance valides (IRVE_itinerance_valide)
- fichier csv des doublons (IRVE_itinerance_doublons)

In [15]:
# consolidation des données
itinerance['doublons_stat_maj'] = itinerance['index'].isin(doublons_stat_maj)
itinerance['doublons_pdc'] = itinerance['index'].isin(doublons_pdc)
itinerance['lignes_a_corriger'] = itinerance['index'].isin(itinerance_5['index'])
itinerance['doublons_a_supprimer'] = itinerance['doublons_stat_maj'] | itinerance['doublons_pdc']
itinerance['lignes_ko'] = itinerance['doublons_a_supprimer'] | itinerance['lignes_a_corriger']
print('total des lignes à corriger : ', itinerance['lignes_a_corriger'].sum())
itinerance_doublons = itinerance[itinerance['doublons_a_supprimer']].reset_index(drop=True)
print('total des doublons à supprimer : ', len(itinerance_doublons))
itinerance_ok = itinerance[~itinerance['lignes_ko']].reset_index(drop=True)
print('nombre de pdc avec controles ok : ', len(itinerance_ok))

total des lignes à corriger :  4126
total des doublons à supprimer :  9372
nombre de pdc avec controles ok :  104761


In [16]:
# génération des fichiers
extension = log['date_irve'] +'.csv'
itinerance_5_full.to_csv('IRVE_itinerance_residuel' + extension)
itinerance.to_csv('IRVE_itinerance_complet' + extension)
itinerance_ok.to_csv('IRVE_itinerance_valide' + extension)
itinerance_doublons.to_csv('IRVE_itinerance_doublons' + extension)
log['IRVE_itinerance_residuel' + extension] = len(itinerance_5_full)
log['IRVE_itinerance_complet' + extension] = len(itinerance)
log['IRVE_itinerance_valide' + extension] = len(itinerance_ok)
log['IRVE_itinerance_valide_stat' + extension] = len(itinerance_ok.drop_duplicates('id_station_itinerance', keep='last'))
log['IRVE_itinerance_doublons' + extension] = len(itinerance_doublons)

In [17]:
l_prefix = list(pd.read_csv(log['chemin'] + '/' + 'Operations_Recharge_16-07-2024.csv', sep=';', low_memory=False, dtype='object')['Identifiant'])
l_prefix[0]

'FR073'

In [18]:
import numpy as np

df = pd.DataFrame({'a':["er", "rt", "", "ert"], 'b': [10, np.nan, 20, 1]})

df['b'] = df['b'].replace(np.nan, 0)
df['a'] = df['a'].replace("", "empty")
df
itinerance_ok['cable_t2_attache'].astype('bool')


0         True
1         True
2         True
3         True
4         True
          ... 
104756    True
104757    True
104758    True
104759    True
104760    True
Name: cable_t2_attache, Length: 104761, dtype: bool

In [19]:
from pydantic import (BaseModel, EmailStr, PastDate, PositiveFloat, PositiveInt, ValidationError)
from pydantic_extra_types.phone_numbers import PhoneNumber
from copy import copy
import re

PREFIX = list(pd.read_csv(log['chemin'] + '/' + 'Operations_Recharge_16-07-2024.csv', sep=';', low_memory=False, dtype='object')['Identifiant'])
    
NUMPDL = re.compile(r"^\d{14}$")

class FrenchPhoneNumber(PhoneNumber):
    """A phone number with french defaults."""
    default_region_code = "FR"
    
class EmailStr(BaseModel):
    x: EmailStr

class PastDate(BaseModel):
    x: PastDate

class PositiveInt(BaseModel):
    x: PositiveInt

class FrenchPhoneNumber(BaseModel):
    x: FrenchPhoneNumber

class NumPdl:
    def __init__(self, x=None):
        self.x = x
        if NUMPDL.fullmatch(x) is None:
            raise ValidationError

def check_value(value, clas=None, default=None):    
    try:
        clas(x=value)
    except:
        return default
    return value

def set_prefix(pref):    
    return pref if pref in PREFIX else PREFIX[0]
    
def pandas_to_qualicharge(data):
    '''
    convertit les champs suivants (et applique une valeur par défaut si erreur) :
        - contact_amenageur: EmailStr ('amenageur@email.fr')
        - contact_operateur: EmailStr ('example@email.fr')
        - telephone_operateur: FrenchPhoneNumber (06-06-06-06-06)
        - nbre_pdc: PositiveInt (0)
        - date_maj: PastDate ('1900-01-01')
        - num_pdl: "^\d{14}$" ('00001234567890')
        - date_mise_en_service: PastDate ('1900-01-01')
        - puissance_nominale: PositiveFloat (0.0)
    et applique le prefixe des stations aux pdc.
    '''
    quali = copy(data)

    quali['paiement_autre'] = quali['paiement_autre'].astype('bool')
    quali['cable_t2_attache'] = quali['cable_t2_attache'].astype('bool')
    quali['raccordement'] = quali['raccordement'].replace(np.nan, 'Direct')
    quali['siren_amenageur'] = quali['siren_amenageur'].astype('string').replace(["", np.nan], "000000000")
    quali['code_insee_commune'] = quali['code_insee_commune'].astype('string').replace(["", np.nan], "99999")
    
    quali['contact_amenageur'] = quali['contact_amenageur'].apply(check_value,  clas=EmailStr, default='amenageur@email.fr')
    quali['contact_operateur'] = quali['contact_operateur'].apply(check_value,  clas=EmailStr, default='operateur@email.fr')
    quali['telephone_operateur'] = quali['telephone_operateur'].apply(check_value,  clas=FrenchPhoneNumber, default='06-06-06-06-06')
    quali['nbre_pdc'] = quali['nbre_pdc'].apply(check_value,  clas=PositiveInt, default=0)
    quali['date_maj'] = quali['date_maj'].apply(check_value,  clas=PastDate, default='1900-01-01')
    quali['num_pdl'] = quali['num_pdl'].apply(check_value,  clas=NumPdl, default='00001234567890')
    quali['date_mise_en_service'] = quali['date_mise_en_service'].apply(check_value,  clas=PastDate, default='1900-01-01')
    quali['puissance_nominale'] = quali['puissance_nominale'].apply(check_value,  clas=PositiveFloat, default=0.01)
    prefix = quali['id_station_itinerance'].astype('str').str.slice(stop=5).apply(set_prefix)
    quali['id_station_itinerance'] = prefix.str.cat(quali['id_station_itinerance'].astype('str').str.slice(start=5))
    quali['id_pdc_itinerance'] = prefix.str.cat(quali['id_pdc_itinerance'].astype('str').str.slice(start=5))

    quali['restriction_gabarit'] = quali['restriction_gabarit'].astype('string')
    return quali

itinerance_quali = pandas_to_qualicharge(itinerance_ok)

In [20]:
type(itinerance_quali.loc[85, 'restriction_gabarit'])
itinerance_quali['paiement_autre'].unique()

array([ True])

In [21]:
itinerance_quali.loc[:80, relations]


,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
0,0,bornes@solveo-energies.com,SOLVEO ENERGIES,"[1.407644, 43.628261]",5 Allee Charles Gandia 31100 Toulouse,FR073E3785539,Golf de Garonne,Parking public,8,Accès libre,Mo-Su 08:00-19:00,false,FR073E3785539,2024-10-24,2024-12-20T06:15:33.880000+00:00
1,1,bornes@solveo-energies.com,SOLVEO ENERGIES,"[1.407644, 43.628261]",5 Allee Charles Gandia 31100 Toulouse,FR073E3785539,Golf de Garonne,Parking public,8,Accès libre,Mo-Su 08:00-19:00,false,FR073E3785590,2024-10-24,2024-12-20T06:15:33.880000+00:00
2,2,bornes@solveo-energies.com,SOLVEO ENERGIES,"[1.407644, 43.628261]",5 Allee Charles Gandia 31100 Toulouse,FR073E3785539,Golf de Garonne,Parking public,8,Accès libre,Mo-Su 08:00-19:00,false,FR073E3785718,2024-10-24,2024-12-20T06:15:33.880000+00:00
3,3,bornes@solveo-energies.com,SOLVEO ENERGIES,"[1.407644, 43.628261]",5 Allee Charles Gandia 31100 Toulouse,FR073E3785539,Golf de Garonne,Parking public,8,Accès libre,Mo-Su 08:00-19:00,false,FR073E3785899,2024-10-24,2024-12-20T06:15:33.880000+00:00
4,4,bornes@solveo-energies.com,SOLVEO ENERGIES,"[1.407644, 43.628261]",5 Allee Charles Gandia 31100 Toulouse,FR073E3785539,Golf de Garonne,Parking public,8,Accès libre,Mo-Su 08:00-19:00,false,FR073E3785900,2024-10-24,2024-12-20T06:15:33.880000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,76,roaming@zunder.com,Zunder,"[0.891121 , 47.301879]","A85, km115, Athée-sur-Cher 37270 France",FR073P1291553774613573967,Zunder/122289,Station dédiée à la recharge rapide,39,Accès libre,24/7,False,FR073EFR3701ER24,2024-12-27,2025-01-01T03:01:18+00:00
77,77,roaming@zunder.com,Zunder,"[0.891121 , 47.301879]","A85, km115, Athée-sur-Cher 37270 France",FR073P1291553774613573967,Zunder/122289,Station dédiée à la recharge rapide,39,Accès libre,24/7,False,FR073EFR3701ER25,2024-12-27,2025-01-01T03:01:18+00:00
78,78,roaming@zunder.com,Zunder,"[0.891121 , 47.301879]","A85, km115, Athée-sur-Cher 37270 France",FR073P1291553774613573967,Zunder/122289,Station dédiée à la recharge rapide,39,Accès libre,24/7,False,FR073EFR3701ER26,2024-12-27,2025-01-01T03:01:18+00:00
79,79,roaming@zunder.com,Zunder,"[0.891121 , 47.301879]","A85, km115, Athée-sur-Cher 37270 France",FR073P1291553774613573967,Zunder/122289,Station dédiée à la recharge rapide,39,Accès libre,24/7,False,FR073EFR3701ER27,2024-12-27,2025-01-01T03:01:18+00:00


In [22]:
itinerance_quali.loc[:, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
0,0,bornes@solveo-energies.com,SOLVEO ENERGIES,"[1.407644, 43.628261]",5 Allee Charles Gandia 31100 Toulouse,FR073E3785539,Golf de Garonne,Parking public,8,Accès libre,Mo-Su 08:00-19:00,false,FR073E3785539,2024-10-24,2024-12-20T06:15:33.880000+00:00
1,1,bornes@solveo-energies.com,SOLVEO ENERGIES,"[1.407644, 43.628261]",5 Allee Charles Gandia 31100 Toulouse,FR073E3785539,Golf de Garonne,Parking public,8,Accès libre,Mo-Su 08:00-19:00,false,FR073E3785590,2024-10-24,2024-12-20T06:15:33.880000+00:00
2,2,bornes@solveo-energies.com,SOLVEO ENERGIES,"[1.407644, 43.628261]",5 Allee Charles Gandia 31100 Toulouse,FR073E3785539,Golf de Garonne,Parking public,8,Accès libre,Mo-Su 08:00-19:00,false,FR073E3785718,2024-10-24,2024-12-20T06:15:33.880000+00:00
3,3,bornes@solveo-energies.com,SOLVEO ENERGIES,"[1.407644, 43.628261]",5 Allee Charles Gandia 31100 Toulouse,FR073E3785539,Golf de Garonne,Parking public,8,Accès libre,Mo-Su 08:00-19:00,false,FR073E3785899,2024-10-24,2024-12-20T06:15:33.880000+00:00
4,4,bornes@solveo-energies.com,SOLVEO ENERGIES,"[1.407644, 43.628261]",5 Allee Charles Gandia 31100 Toulouse,FR073E3785539,Golf de Garonne,Parking public,8,Accès libre,Mo-Su 08:00-19:00,false,FR073E3785900,2024-10-24,2024-12-20T06:15:33.880000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104756,119275,advenir@zeborne.com,TOYOTA NICE - NOVELLIPSE,"[7.201547, 43.6885902]",108 Avenue Simone Veil 06200 Nice,FRZTLE24DC55361,Toyota - Nice - 24kW DC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:30,Mo-Fr 13:30–19:00,Sat 09:00-...",false,FRZTLE24DC55361,2021-02-24,2024-12-06T06:44:53.447000+00:00
104757,119276,advenir@zeborne.com,GARAGE NELLO CHELI - BEAUNE,"[4.86082418, 47.0236522]",RUE LUCIEN PERRIAUX ZI DE VIGNOLES 21200 Beaune,FRZTLE24DC61318,Toyota - Beaune - 24kW DC,Parking public,1,Accès libre,"Mo-Fr 07:45-12:00,Mo-Fr 13:45-19:00,Sat 09:00-...",false,FRZTLE24DC61318,2021-05-24,2024-12-06T06:45:00.492000+00:00
104758,119277,advenir@zeborne.com,GARAGE NELLO CHELI - BEAUNE,"[4.86082418, 47.0236522]",RUE LUCIEN PERRIAUX ZI DE VIGNOLES 21200 Beaune,FRZTLE24DC61319,Toyota - Beaune - 24kW DC,Parking public,1,Accès libre,"Mo-Fr 07:45-12:00,Mo-Fr 13:45-19:00,Sat 09:00-...",false,FRZTLE24DC61319,2021-05-24,2024-12-06T06:45:00.492000+00:00
104759,119278,advenir@zeborne.com,TOYOTA ANNEMASSE - DEGENEVE,"[6.2174783, 46.1858677]",8 RUE DE L'INDUSTRIE 74240 Gaillard,FRZTLE24DC63446,Toyota - Annemasse - 24kW DC,Parking public,1,Accès libre,"Mo-Fr 08:00-12:00,Mo-Fr 14:00-19:00,Sat 09:00-...",false,FRZTLE24DC63446,2021-06-16,2024-12-06T06:45:00.804000+00:00


In [23]:
import csv
itinerance_quali.to_csv('IRVE_itinerance_valide_quali' + extension, index=False, quoting=csv.QUOTE_ALL, na_rep='empty')

In [24]:
itinerance_quali['paiement_autre'].to_csv('test.csv', index=False, quoting=csv.QUOTE_ALL, na_rep='empty')

In [25]:
for op in oper_5:
    print('IRVE_itinerance_residuel' + '_' + op + '_' + extension)
    itinerance_5_full.loc[itinerance_5_full.contact_operateur == op].to_csv('IRVE_itinerance_residuel' + '_' + op + '_' + extension)

IRVE_itinerance_residuel_support@alizecharge.fr_2025-01-03.csv
IRVE_itinerance_residuel_supervision-ev.france@totalenergies.com_2025-01-03.csv
IRVE_itinerance_residuel_sav@izivia.com_2025-01-03.csv
IRVE_itinerance_residuel_a.buffler@bouygues-es.com_2025-01-03.csv
IRVE_itinerance_residuel_cpo@citeos.com_2025-01-03.csv


### vérification de l'intégrité

In [26]:
# vérification de l'absence d'erreurs
res = analyse_integrite(itinerance_ok.loc[:, relations], schema, affiche=False)
log['bilan_erreurs'] = sum(res.values())
log['date'] = datetime.now().isoformat()
print('bilan intégrité :')
print('    erreurs : ', log['bilan_erreurs'])

bilan intégrité :
    erreurs :  0


In [27]:
# structure des données bonnes
#print(Cdataset(itinerance_ok.loc[:, mandatory]).tree())
print(itinerance_ok.loc[:, mandatory].npd.analysis().tree())

-1: root-derived (104761)
   4 : id_station_itineranc (58956 - 45805)
      0 : contact_operateur (45632 - 173)
      1 : nom_enseigne (42257 - 3548)
      2 : coordonneesXY (18635 - 27170)
         3 : adresse_station (650 - 26520)
      5 : nom_station (18358 - 27447)
      6 : implantation_station (45800 - 5)
      7 : nbre_pdc (45739 - 66)
      8 : condition_acces (45803 - 2)
      9 : horaires (44995 - 810)
      10: station_deux_roues (45799 - 6)
      18: paiement_acte (45797 - 8)
      22: restriction_gabarit (45695 - 110)
   11: id_pdc_itinerance (0 - 104761)
   12: puissance_nominale (104614 - 147)
   13: prise_type_ef (104753 - 8)
   14: prise_type_2 (104753 - 8)
   15: prise_type_combo_ccs (104754 - 7)
   16: prise_type_chademo (104754 - 7)
   17: prise_type_autre (104754 - 7)
   19: paiement_autre (104752 - 9)
   20: reservation (104754 - 7)
   21: accessibilite_pmr (104757 - 4)
   23: date_maj (104051 - 710)
   24: last_modified (104250 - 511)


### Indicateurs

In [28]:
# génération des indicateurs liés aux règles
regles = ['Pdc non unique', 'Station multi-operateurs', 'Station multi-enseignes', 'Station multi-localisations', 
          'Pdc multi-stations', 'station avec plusieurs noms', 'station multi-implantations', 
          'nombre de pdc par station incoherent', 'station multi-acces', 'station multi-horaires', 
          'acces deux-roues incoherent', 'localisation multi-adresses']
principal = [16, 17, 18, 19, 20]
secondaire = [21, 22, 23, 24, 25, 26, 27]
irve = itinerance_5_full # residuel
total = len(irve)
indic = {}

irve['principal'] = True
for ind in principal:
    irve['principal'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] = True
for ind in secondaire:
    irve['secondaire'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] |= (~irve['principal'] & ~irve['secondaire'])
irve['verif'] = irve['principal'] & irve['secondaire']
indic['principal pdc'] = int(total - irve['principal'].sum())
indic['secondaire pdc'] = int(total - irve['secondaire'].sum())

irve_p = irve[~irve['principal']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
irve_s = irve[~irve['secondaire']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
indic['principal stat'] = len(irve_p)
indic['secondaire stat'] = len(irve_s)

In [29]:
# ajout des principaux opérateurs contributeur de défauts
indic['operateurs'] = list(oper_5)
indic['erreurs_operateurs'] = list(err_op_5)

In [30]:
# ajout de l'indicateur d'écart entre nombre de pdc calculés et saisis
add_nbre_pdc(irve, affiche=True)
indic['nb_pdc_calc-nbre_pdc'] = int(irve['calc-nbre'].sum())

nbre_pdc_ko           1472
36  %


In [31]:
# indicateur d'écart entre nombre de pdc calculés et saisis sur les données sans doublon
itinerance_sans_doublons = itinerance[~itinerance['doublons_a_supprimer']].reset_index(drop=True)
add_nbre_pdc(itinerance_sans_doublons, affiche=True)

nbre_pdc_ko          32415
30  %


In [32]:
len(itinerance_4)

4234

In [33]:
# stockage des indicateurs
log |= indic
log_file = log['chemin'] + '/' + 'logfile.txt'
#with open('logfile.txt', 'a', encoding="utf-8") as f:
with open(log_file, 'a', encoding="utf-8") as f:
    f.write(json.dumps(log) + '\n')

In [34]:
itinerance_5_full.columns[16:28]

Index(['index - id_pdc_itinerance',
       'contact_operateur - id_station_itinerance',
       'nom_enseigne - id_station_itinerance',
       'coordonneesXY - id_station_itinerance',
       'id_station_itinerance - id_pdc_itinerance',
       'nom_station - id_station_itinerance',
       'implantation_station - id_station_itinerance',
       'nbre_pdc - id_station_itinerance',
       'condition_acces - id_station_itinerance',
       'horaires - id_station_itinerance',
       'station_deux_roues - id_station_itinerance',
       'adresse_station - coordonneesXY'],
      dtype='object')

-----------------------
## 8 - Exemples d'erreurs résiduelles 

### Erreurs multiples 
- exemple avec le nombre maximal d'erreurs (3) (109 pdc)
- exemple station 'FR55CP92140' avec 29 pdc ! : 5 coordonnées XY, 5 nom_station, 2 nbre_pdc)

In [35]:
maxi.loc[:, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1451,60254,support@mobilize-ps.fr,JRA SAUMUR,"[-0.059981, 47.274681]","\tBd Vallerots, 49400 Saumur",FRMBZEEVHJW,JRA SAUMUR,Parking privé à usage public,1,Accès libre,"Mo-Fr 07:30-12:00,Mo-Fr 14:00-19:00",False,FRMBZEEVHJW1,2024-07-10,2024-11-15T16:46:22.342000+00:00
1452,60255,support@mobilize-ps.fr,JRA Saumur,"[-0.05996536452405448, 47.27462246981463]","Bd Vallerots, 49400 Saumur",FRMBZEEVHJW,JRA Saumur,Parking privé à usage public,1,Accès libre,24/7,False,FRMBZEEVHJW2,2024-07-10,2024-11-15T16:46:22.342000+00:00
1462,60298,support@mobilize-ps.fr,BODEMER BAYEUX,"[-0.700226002831211, 49.27084281571326]","16 Bd Sadi Carnot, 14400 Bayeux",FRMBZESMYQC,BODEMER Bayeux,Parking privé à usage public,1,Accès libre,24/7,False,FRMBZESMYCQ1,2024-06-03,2024-11-15T16:46:22.342000+00:00
1463,60299,support@mobilize-ps.fr,BODEMER - SCAUTO BAYEUX,"[-0.700259, 49.270901]","\t16 Bd Sadi Carnot, 14400 Bayeux",FRMBZESMYQC,\t\tScauto Bayeux,Parking privé à usage public,1,Accès libre,"Mo-Fr 07:30-12:00,Mo-Fr 14:00-19:00",False,FRMBZESMYQC1,2024-06-03,2024-11-15T16:46:22.342000+00:00


In [36]:
itinerance_4.loc[itinerance_4.id_station_itinerance == 'FR55CP92140', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### Repartition des defauts par operateur
Répartition pour les 10 opérateurs présentant le plus de défauts

In [73]:
print('PdC avec une incohérence : ')
for operateur in oper_10:
    print('\n', sum(itinerance_5.contact_operateur == operateur))
    print('Répartition des incohérences ', operateur, ' :')
    for cont in controles:
        #print(cont, sum((itinerance_4['contact_operateur'] == 'supervision-ev.france@totalenergies.com') & ~(itinerance_4[cont])))
        print('{:<50} {:>5}'.format(cont, sum((itinerance_4['contact_operateur'] == operateur) & ~(itinerance_4[cont]))))

PdC avec une incohérence : 

 799
Répartition des incohérences  support@alizecharge.fr  :
index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              0
nom_enseigne - id_station_itinerance                   0
coordonneesXY - id_station_itinerance                205
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  357
implantation_station - id_station_itinerance          79
nbre_pdc - id_station_itinerance                     335
condition_acces - id_station_itinerance                0
horaires - id_station_itinerance                       0
station_deux_roues - id_station_itinerance             0
adresse_station - coordonneesXY                      447

 627
Répartition des incohérences  supervision-ev.france@totalenergies.com  :
index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              0
nom_enseigne - id_station_itinera

### Defauts operateur ALIZECHARGE
Les défauts identifiés concernent des stations utilisant un même identifiant. Par exemple, la station 'FRADPP91479018' contient 82 points de recharge avec plusieurs adresses et un nombre de points de rechage variable. Cet exemple concerne des stations implantées sur plusieurs étages d'un parking (on devrait donc a minima identifier une station par étage).

In [38]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'support@alizecharge.fr', relations][50:200]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
227,2416,support@alizecharge.fr,ADP Groupe,"[2.610949,49.0076438]","Route Périphérique Sud, Base Arrière Taxi, 77...",FRADPE77294001,CDG - BAT (Réservé exclusivement aux taxis par...,Parking public,20,Accès libre,Mo-Su 00:00-23:57,False,FRADPE77294001281,2024-12-31,2024-12-31T11:18:24.916000+00:00
228,2417,support@alizecharge.fr,ADP Groupe,"[2.610949,49.0076438]","Route Périphérique Sud, Base Arrière Taxi, 77...",FRADPE77294001,CDG - BAT (Réservé exclusivement aux taxis par...,Parking public,20,Accès libre,Mo-Su 00:00-23:57,False,FRADPE77294001283,2024-12-31,2024-12-31T11:18:24.916000+00:00
229,2418,support@alizecharge.fr,ADP Groupe,"[2.363758,48.733494]","Rue de la Fossette, 94390 Paray-Vieille-Poste",FRADPE77294001,ORY - BAT (Réservé exclusivement aux taxis par...,Parking public,26,Accès libre,Mo-Su 00:00-23:57,False,FRADPE7729400130,2024-12-31,2024-12-31T11:18:24.916000+00:00
230,2419,support@alizecharge.fr,ADP Groupe,"[2.363758,48.733494]","Rue de la Fossette, 94390 Paray-Vieille-Poste",FRADPE77294001,ORY - BAT (Réservé exclusivement aux taxis par...,Parking public,26,Accès libre,Mo-Su 00:00-23:57,False,FRADPE7729400131,2024-12-31,2024-12-31T11:18:24.916000+00:00
231,2420,support@alizecharge.fr,ADP Groupe,"[2.363758,48.733494]","Rue de la Fossette, 94390 Paray-Vieille-Poste",FRADPE77294001,ORY - BAT (Réservé exclusivement aux taxis par...,Parking public,26,Accès libre,Mo-Su 00:00-23:57,False,FRADPE7729400132,2024-12-31,2024-12-31T11:18:24.916000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163,57540,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.72939,45.18938]","Place Jean Achard, 38000 GRENOBLE",FRM38E38185001,GRENOBLE - Place Jean Achard,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRM38E381850011,2024-12-31,2024-12-31T11:18:24.916000+00:00
1164,57541,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.72939,45.18938]","Place Jean Achard, 38000 GRENOBLE",FRM38E38185001,GRENOBLE - Place Jean Achard,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRM38E381850012,2024-12-31,2024-12-31T11:18:24.916000+00:00
1165,57542,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.72949,45.18938]","Place Jean Achard (CITIZ), 38000 GRENOBLE",FRM38E38185001,GRENOBLE - Place Jean Achard (CITIZ),Voirie,1,Accès libre,Mo-Su 00:00-23:57,False,FRM38E381850013,2024-12-31,2024-12-31T11:18:24.916000+00:00
1166,57546,support@alizecharge.fr,Grenoble-Alpes Métropole,"[5.704011,45.1844]","Rue Victor Lastella, 38000 GRENOBLE",FRM38E38185005,GRENOBLE - Rue Victor Lastella - Chambre des M...,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRM38E381850051,2024-12-31,2024-12-31T11:18:24.916000+00:00


In [39]:
# example [2.362554,48.728787]
adp = itinerance_4.loc[itinerance_4.coordonneesXY == '[2.362554,48.728787]']
adresses = list(adp.drop_duplicates('adresse_station')['adresse_station'])
print(adresses)

print(adp.loc[~adp['nom_station - id_station_itinerance'], ['nom_station', 'id_station_itinerance']])

print(adp.loc[~adp['implantation_station - id_station_itinerance'], ['implantation_station', 'id_station_itinerance']])

print(adp.loc[~adp['nbre_pdc - id_station_itinerance'], ['nbre_pdc', 'id_station_itinerance']])

itinerance_4.loc[itinerance_4.coordonneesXY == '[2.362554,48.728787]', relations]

['2 Rue des Transporteurs, 94390 Paray-Vieille-Poste', '2 Rue des Transporteurs, 94390  Paray-Vieille-Poste']
                          nom_station id_station_itinerance
246  ORY - Parking P3 SILO - Niveau 2        FRADPE91479018
247  ORY - Parking P3 SILO - Niveau 2        FRADPE91479018
248  ORY - Parking P3 SILO - Niveau 2        FRADPE91479018
249  ORY - Parking P3 SILO - Niveau 2        FRADPE91479018
250  ORY - Parking P3 SILO - Niveau 2        FRADPE91479018
..                                ...                   ...
323  ORY - Parking P3 SILO - Niveau 2        FRADPE91479018
324  ORY - Parking P3 SILO - Niveau 6        FRADPE91479018
325  ORY - Parking P3 SILO - Niveau 6        FRADPE91479018
326  ORY - Parking P3 SILO - Niveau 6        FRADPE91479018
327  ORY - Parking P3 SILO - Niveau 2        FRADPE91479018

[82 rows x 2 columns]
Empty DataFrame
Columns: [implantation_station, id_station_itinerance]
Index: []
    nbre_pdc id_station_itinerance
246       16        FRADPE91479

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
246,2505,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE91479018,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE914790181,2024-12-31,2024-12-31T11:18:24.916000+00:00
247,2506,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE91479018,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901810,2024-12-31,2024-12-31T11:18:24.916000+00:00
248,2507,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE91479018,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901811,2024-12-31,2024-12-31T11:18:24.916000+00:00
249,2508,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE91479018,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901812,2024-12-31,2024-12-31T11:18:24.916000+00:00
250,2509,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE91479018,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901813,2024-12-31,2024-12-31T11:18:24.916000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,2582,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE91479018,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE914790188,2024-12-31,2024-12-31T11:18:24.916000+00:00
324,2583,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE91479018,ORY - Parking P3 SILO - Niveau 6,Parking public,18,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901880,2024-12-31,2024-12-31T11:18:24.916000+00:00
325,2584,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE91479018,ORY - Parking P3 SILO - Niveau 6,Parking public,18,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901881,2024-12-31,2024-12-31T11:18:24.916000+00:00
326,2585,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE91479018,ORY - Parking P3 SILO - Niveau 6,Parking public,18,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901882,2024-12-31,2024-12-31T11:18:24.916000+00:00


In [40]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'support@alizecharge.fr', relations].loc[~itinerance_4['adresse_station - coordonneesXY']][400:700]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2789,94656,support@alizecharge.fr,Ecocharge77,"[2.746701,48.451259]","Route du Cèpe, 77920 SAMOIS-SUR-SEINE",FRS77E77441001,SAMOIS-SUR-SEINE - Cèpe,Voirie,4,Accès libre,Mo-Su 00:00-23:57,False,FRS77E774410014,2024-12-31,2024-12-31T11:18:24.916000+00:00
2790,94657,support@alizecharge.fr,Ecocharge77,"[2.961653,48.885735]","Rue De Meaux, 77580 SANCY",FRS77E77443001,SANCY - Rue De Meaux,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E774430011,2024-12-31,2024-12-31T11:18:24.916000+00:00
2791,94658,support@alizecharge.fr,Ecocharge77,"[2.961653,48.885735]","Rue De Meaux, 77580 SANCY",FRS77E77443001,SANCY - Rue De Meaux,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E774430012,2024-12-31,2024-12-31T11:18:24.916000+00:00
2792,94659,support@alizecharge.fr,Ecocharge77,"[2.582113,48.596577]","Avenue Léon Blum, 77176 SAVIGNY-LE-TEMPLE",FRS77E77445001,SAVIGNY-LE-TEMPLE - Avenue Léon Blum,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E774450011,2024-12-31,2024-12-31T11:18:24.916000+00:00
2793,94660,support@alizecharge.fr,Ecocharge77,"[2.582113,48.596577]","Avenue Léon Blum, 77176 SAVIGNY-LE-TEMPLE",FRS77E77445001,SAVIGNY-LE-TEMPLE - Avenue Léon Blum,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E774450012,2024-12-31,2024-12-31T11:18:24.916000+00:00
2794,94661,support@alizecharge.fr,Ecocharge77,"[2.549188,48.557755]","Rue De Melun, 77240 SEINE-PORT",FRS77E77447001,SEINE-PORT - Rue De Melun,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E7744700111,2024-12-31,2024-12-31T11:18:24.916000+00:00
2795,94662,support@alizecharge.fr,Ecocharge77,"[2.549188,48.557755]","Rue De Melun, 77240 SEINE-PORT",FRS77E77447001,SEINE-PORT - Rue De Melun,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E7744700112,2024-12-31,2024-12-31T11:18:24.916000+00:00
2796,94669,support@alizecharge.fr,Ecocharge77,"[3.289959,48.509865]","Rue De La Mairie, 77650 SOISY-BOUY",FRS77E77456001,SOISY-BOUY - Rue De La Mairie,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E774560011,2024-12-31,2024-12-31T11:18:24.916000+00:00
2797,94670,support@alizecharge.fr,Ecocharge77,"[3.289959,48.509865]","Rue De La Mairie, 77650 SOISY-BOUY",FRS77E77456001,SOISY-BOUY - Rue De La Mairie,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E774560012,2024-12-31,2024-12-31T11:18:24.916000+00:00
2798,94671,support@alizecharge.fr,Ecocharge77,"[2.717254,48.657776]","Rue De La Tournelle, 77111 SOLERS",FRS77E77457001,SOLERS - Rue De La Tournelle,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E774570011,2024-12-31,2024-12-31T11:18:24.916000+00:00


In [41]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[2.969121,48.282539]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2832,94723,support@alizecharge.fr,Ecocharge77,"[2.969121,48.282539]","Rue Grande, 77940 VOULX",FRS77E77531001,VOULX - Rue Grande,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E775310011,2024-12-31,2024-12-31T11:18:24.916000+00:00
2833,94724,support@alizecharge.fr,Ecocharge77,"[2.969121,48.282539]","Rue Grande, 77940 VOULX",FRS77E77531001,VOULX - Rue Grande,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E775310012,2024-12-31,2024-12-31T11:18:24.916000+00:00
2950,94875,a.buffler@bouygues-es.com,Ecocharge77,"[2.969121,48.282539]",Rue Grande,FRS77P77531A,Voulx - Grande rue,Voirie,2,Accès libre,24/7,false,FRS77EEC7716711,2024-12-24,2024-12-24T09:00:19.626000+00:00
2951,94876,a.buffler@bouygues-es.com,Ecocharge77,"[2.969121,48.282539]",Rue Grande,FRS77P77531A,Voulx - Grande rue,Voirie,2,Accès libre,24/7,false,FRS77EEC7716712,2024-12-24,2024-12-24T09:00:19.626000+00:00


### Defauts operateur IZIVIA
Les défauts au nombre de 280 sont liés à une incohérence entre adresse et coordonnées (une même coordonnée a plusieurs adresses différentes). Ce défaut est lié soit à des erreurs de saisie, soit à des ajouts (ou remplacements) ultérieurs de stations à une même localisation.

Les stations Izivia en erreur ont un id_station_itinerance identique à l'id_pdc_itinerance, ce qui se traduit par des adresses multiples pour une même coordonnée.

Par exemple, sur un parking [-0.530598, 47.398424], on trouve 17 stations avec la même coordonnée, le même identifiant entre point de recharge et station et un nombre de PdC indiqué entre 1 et 3.

In [42]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'sav@izivia.com', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
426,15407,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24711,HYPER U - MURS-ERIGNE,Voirie,2,Accès libre,24/7,false,FRE04EOAZS24711,2025-01-02,2025-01-02T07:32:27.286000+00:00
427,15408,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24712,HYPER U - MURS-ERIGNE,Voirie,2,Accès libre,24/7,false,FRE04EOAZS24712,2025-01-02,2025-01-02T07:32:27.286000+00:00
428,15409,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24721,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24721,2025-01-02,2025-01-02T07:32:27.286000+00:00
429,15410,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24722,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24722,2025-01-02,2025-01-02T07:32:27.286000+00:00
430,15411,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24723,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24723,2025-01-02,2025-01-02T07:32:27.286000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3646,110593,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1622,AEROVILLE - TERMINAL COOK,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1622,2025-01-02,2025-01-02T07:32:25.774000+00:00
3647,110594,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1631,AEROVILLE - TERMINAL COOK,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1631,2025-01-02,2025-01-02T07:32:25.774000+00:00
3648,110595,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL DAEROVILLE 95700 ROISSY-EN-F...,FRURWPUNIB1632,AEROVILLE - TERMINAL COOK,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1632,2025-01-02,2025-01-02T07:32:25.774000+00:00
3649,110596,sav@izivia.com,UNIBAIL,"[2.523685, 48.9908]",CENTRE COMMERCIAL AEROVILLE 95700 ROISSY-EN-FR...,FRURWPUNIB1711,AEROVILLE - TOKYO,Voirie,2,Accès libre,24/7,false,FRURWEUNIB1711,2025-01-02,2025-01-02T07:32:25.774000+00:00


In [43]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-0.530598, 47.398424]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
426,15407,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24711,HYPER U - MURS-ERIGNE,Voirie,2,Accès libre,24/7,false,FRE04EOAZS24711,2025-01-02,2025-01-02T07:32:27.286000+00:00
427,15408,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24712,HYPER U - MURS-ERIGNE,Voirie,2,Accès libre,24/7,false,FRE04EOAZS24712,2025-01-02,2025-01-02T07:32:27.286000+00:00
428,15409,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24721,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24721,2025-01-02,2025-01-02T07:32:27.286000+00:00
429,15410,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24722,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24722,2025-01-02,2025-01-02T07:32:27.286000+00:00
430,15411,sav@izivia.com,IZIVIA EXPRESS,"[-0.530598, 47.398424]",26 RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FRE04POAZS24723,HYPER U - MURS-ERIGNE,Voirie,3,Accès libre,24/7,false,FRE04EOAZS24723,2025-01-02,2025-01-02T07:32:27.286000+00:00
1735,65304,sav@izivia.com,SYSTEME U,"[-0.530598, 47.398424]",RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FROTHPOTHR372101,HYPER U - MURS-ERIGNE,Voirie,1,Accès libre,24/7,false,FROTHEOTHR372101,2025-01-02,2025-01-02T07:32:59.358000+00:00
1736,65305,sav@izivia.com,SYSTEME U,"[-0.530598, 47.398424]",RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FROTHPOTHR37211,HYPER U - MURS-ERIGNE,Voirie,1,Accès libre,24/7,false,FROTHEOTHR37211,2025-01-02,2025-01-02T07:32:59.358000+00:00
1737,65306,sav@izivia.com,SYSTEME U,"[-0.530598, 47.398424]",RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FROTHPOTHR372111,HYPER U - MURS-ERIGNE,Voirie,1,Accès libre,24/7,false,FROTHEOTHR372111,2025-01-02,2025-01-02T07:32:59.358000+00:00
1738,65307,sav@izivia.com,SYSTEME U,"[-0.530598, 47.398424]",RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FROTHPOTHR372121,HYPER U - MURS-ERIGNE,Voirie,1,Accès libre,24/7,false,FROTHEOTHR372121,2025-01-02,2025-01-02T07:32:59.358000+00:00
1739,65308,sav@izivia.com,SYSTEME U,"[-0.530598, 47.398424]",RUE VALENTIN DES ORMEAUX 49610 MURS-ERIGNE,FROTHPOTHR37221,HYPER U - MURS-ERIGNE,Voirie,1,Accès libre,24/7,false,FROTHEOTHR37221,2025-01-02,2025-01-02T07:32:59.358000+00:00


### Defauts operateur CITEOS
Les défauts (230 points de recharge) sont liés à une incohérence entre adresse et coordonnées (une même coordonnée a plusieurs adresses différentes). Ce défaut est lié principalement à des doublons de pdc entre Citeos et Izivia, ce qui se traduit par des adresses multiples pour une même coordonnée.

Par exemple, à Soulac sur mer, à la coordonnée [-1.122837, 45.515089], on trouve 1 station Citeos Mobive avec 3 points de recharge et également 1 station Izivia Mobive avec 3 points de recharge.

In [74]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'cpo@citeos.com', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1215,57882,cpo@citeos.com,CPO CITEOS Mobive,"[1.570625, 45.442994]",31 Av. de la Gare 19140 UZERCHE,FRMBIPVAEKR,MOBIVE | Uzerche | Av. de la Gare,Voirie,2,Accès libre,24/7,false,FRMBIEVAEKR1,2024-10-31,2025-01-02T10:48:40.634000+00:00
1216,57883,cpo@citeos.com,CPO CITEOS Mobive,"[1.570625, 45.442994]",31 Av. de la Gare 19140 UZERCHE,FRMBIPVAEKR,MOBIVE | Uzerche | Av. de la Gare,Voirie,2,Accès libre,24/7,false,FRMBIEVAEKR2,2024-10-31,2025-01-02T10:48:40.634000+00:00
1217,57898,cpo@citeos.com,CPO CITEOS Mobive,"[1.406172, 45.953075]",Rue de la Gare 87240 AMBAZAC,FRMBIPVAEUM,MOBIVE | Ambazac | Gare,Voirie,2,Accès libre,24/7,false,FRMBIEVAEUM1,2024-11-05,2025-01-02T10:48:40.634000+00:00
1218,57899,cpo@citeos.com,CPO CITEOS Mobive,"[1.406172, 45.953075]",Rue de la Gare 87240 AMBAZAC,FRMBIPVAEUM,MOBIVE | Ambazac | Gare,Voirie,2,Accès libre,24/7,false,FRMBIEVAEUM2,2024-11-05,2025-01-02T10:48:40.634000+00:00
1219,57950,cpo@citeos.com,CPO CITEOS Mobive,"[0.028132, 45.894077]",15 Route Des Cagouilles 16140 AIGRE,FRMBIPVAFXA,MOBIVE | Aigre | Villejésus | Place de L'Glise,Voirie,2,Accès libre,24/7,false,FRMBIEVAFXA1,2024-10-31,2025-01-02T10:48:40.634000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444,60068,cpo@citeos.com,CPO CITEOS Mobive,"[-1.122837, 45.515089]",3 Rue de L'Hôtel de Ville 33780 SOULAC-SUR-MER,FRMBIPVCCRR,MOBIVE | Soulac-Sur-Mer | Parking de la Mairie,Voirie,3,Accès libre,24/7,false,FRMBIEVCCRR2,2024-11-05,2025-01-02T10:48:40.634000+00:00
1445,60069,cpo@citeos.com,CPO CITEOS Mobive,"[-1.122837, 45.515089]",3 Rue de L'Hôtel de Ville 33780 SOULAC-SUR-MER,FRMBIPVCCRR,MOBIVE | Soulac-Sur-Mer | Parking de la Mairie,Voirie,3,Accès libre,24/7,false,FRMBIEVCCRR3,2024-11-05,2025-01-02T10:48:40.634000+00:00
1446,60075,cpo@citeos.com,CPO CITEOS Mobive,"[-0.826756, 44.664279]",3 Avenue ZAC 2000 33380 MIOS,FRMBIPVCCXF,MOBIVE | MIOS - Echangeur Pelouneyres,Voirie,3,Accès libre,24/7,false,FRMBIEVCCXF1,2024-11-05,2025-01-02T10:48:40.634000+00:00
1447,60076,cpo@citeos.com,CPO CITEOS Mobive,"[-0.826756, 44.664279]",3 Avenue ZAC 2000 33380 MIOS,FRMBIPVCCXF,MOBIVE | MIOS - Echangeur Pelouneyres,Voirie,3,Accès libre,24/7,false,FRMBIEVCCXF2,2024-11-05,2025-01-02T10:48:40.634000+00:00


In [75]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[1.570625, 45.442994]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1215,57882,cpo@citeos.com,CPO CITEOS Mobive,"[1.570625, 45.442994]",31 Av. de la Gare 19140 UZERCHE,FRMBIPVAEKR,MOBIVE | Uzerche | Av. de la Gare,Voirie,2,Accès libre,24/7,false,FRMBIEVAEKR1,2024-10-31,2025-01-02T10:48:40.634000+00:00
1216,57883,cpo@citeos.com,CPO CITEOS Mobive,"[1.570625, 45.442994]",31 Av. de la Gare 19140 UZERCHE,FRMBIPVAEKR,MOBIVE | Uzerche | Av. de la Gare,Voirie,2,Accès libre,24/7,false,FRMBIEVAEKR2,2024-10-31,2025-01-02T10:48:40.634000+00:00
2204,84817,sav@izivia.com,MOBIVE - FDEE 19,"[1.570625, 45.442994]",AVENUE DE LA GARE 19140 UZERCHE,FRS19PMB193111,UZERCHE - AVENUE DE LA GARE,Voirie,2,Accès libre,24/7,false,FRS19EMB193111,2024-05-15,2024-05-15T08:30:02.806000+00:00
2205,84818,sav@izivia.com,MOBIVE - FDEE 19,"[1.570625, 45.442994]",AVENUE DE LA GARE 19140 UZERCHE,FRS19PMB193112,UZERCHE - AVENUE DE LA GARE,Voirie,2,Accès libre,24/7,false,FRS19EMB193112,2024-05-15,2024-05-15T08:30:02.806000+00:00


In [76]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-1.122837, 45.515089]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1443,60067,cpo@citeos.com,CPO CITEOS Mobive,"[-1.122837, 45.515089]",3 Rue de L'Hôtel de Ville 33780 SOULAC-SUR-MER,FRMBIPVCCRR,MOBIVE | Soulac-Sur-Mer | Parking de la Mairie,Voirie,3,Accès libre,24/7,false,FRMBIEVCCRR1,2024-11-05,2025-01-02T10:48:40.634000+00:00
1444,60068,cpo@citeos.com,CPO CITEOS Mobive,"[-1.122837, 45.515089]",3 Rue de L'Hôtel de Ville 33780 SOULAC-SUR-MER,FRMBIPVCCRR,MOBIVE | Soulac-Sur-Mer | Parking de la Mairie,Voirie,3,Accès libre,24/7,false,FRMBIEVCCRR2,2024-11-05,2025-01-02T10:48:40.634000+00:00
1445,60069,cpo@citeos.com,CPO CITEOS Mobive,"[-1.122837, 45.515089]",3 Rue de L'Hôtel de Ville 33780 SOULAC-SUR-MER,FRMBIPVCCRR,MOBIVE | Soulac-Sur-Mer | Parking de la Mairie,Voirie,3,Accès libre,24/7,false,FRMBIEVCCRR3,2024-11-05,2025-01-02T10:48:40.634000+00:00
2296,87275,sav@izivia.com,MOBIVE - SDEEG 33,"[-1.122837, 45.515089]",2 RUE DE LHOTEL DE VILLE 33780 SOULAC-SUR-MER,FRS33PMB3317621,SOULAC-SUR-MER - PARKING DE LA MAIRIE,Voirie,3,Accès libre,24/7,false,FRS33EMB3317621,2024-05-15,2024-05-15T08:30:10.714000+00:00
2297,87276,sav@izivia.com,MOBIVE - SDEEG 33,"[-1.122837, 45.515089]",2 RUE DE LHOTEL DE VILLE 33780 SOULAC-SUR-MER,FRS33PMB3317622,SOULAC-SUR-MER - PARKING DE LA MAIRIE,Voirie,3,Accès libre,24/7,false,FRS33EMB3317622,2024-05-15,2024-05-15T08:30:10.714000+00:00
2298,87277,sav@izivia.com,MOBIVE - SDEEG 33,"[-1.122837, 45.515089]",2 RUE DE LHOTEL DE VILLE 33780 SOULAC-SUR-MER,FRS33PMB3317623,SOULAC-SUR-MER - PARKING DE LA MAIRIE,Voirie,3,Accès libre,24/7,false,FRS33EMB3317623,2024-05-15,2024-05-15T08:30:10.714000+00:00


In [77]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-0.826756, 44.664279]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1446,60075,cpo@citeos.com,CPO CITEOS Mobive,"[-0.826756, 44.664279]",3 Avenue ZAC 2000 33380 MIOS,FRMBIPVCCXF,MOBIVE | MIOS - Echangeur Pelouneyres,Voirie,3,Accès libre,24/7,false,FRMBIEVCCXF1,2024-11-05,2025-01-02T10:48:40.634000+00:00
1447,60076,cpo@citeos.com,CPO CITEOS Mobive,"[-0.826756, 44.664279]",3 Avenue ZAC 2000 33380 MIOS,FRMBIPVCCXF,MOBIVE | MIOS - Echangeur Pelouneyres,Voirie,3,Accès libre,24/7,false,FRMBIEVCCXF2,2024-11-05,2025-01-02T10:48:40.634000+00:00
1448,60077,cpo@citeos.com,CPO CITEOS Mobive,"[-0.826756, 44.664279]",3 Avenue ZAC 2000 33380 MIOS,FRMBIPVCCXF,MOBIVE | MIOS - Echangeur Pelouneyres,Voirie,3,Accès libre,24/7,false,FRMBIEVCCXF3,2024-11-05,2025-01-02T10:48:40.634000+00:00
2276,87250,sav@izivia.com,MOBIVE - SDEEG 33,"[-0.826756, 44.664279]",ECHANGEUR PELOUNEYRES 33380 MIOS,FRS33PMB3315511,MIOS - ECHANGEUR PELOUNEYRES,Voirie,4,Accès libre,24/7,false,FRS33EMB3315511,2024-05-15,2024-05-15T08:30:10.714000+00:00
2277,87251,sav@izivia.com,MOBIVE - SDEEG 33,"[-0.826756, 44.664279]",ECHANGEUR PELOUNEYRES 33380 MIOS,FRS33PMB3315512,MIOS - ECHANGEUR PELOUNEYRES,Voirie,4,Accès libre,24/7,false,FRS33EMB3315512,2024-05-15,2024-05-15T08:30:10.714000+00:00
2278,87252,sav@izivia.com,MOBIVE - SDEEG 33,"[-0.826756, 44.664279]",ECHANGEUR PELOUNEYRES 33380 MIOS,FRS33PMB3315513,MIOS - ECHANGEUR PELOUNEYRES,Voirie,4,Accès libre,24/7,false,FRS33EMB3315513,2024-05-15,2024-05-15T08:30:10.714000+00:00
2279,87253,sav@izivia.com,MOBIVE - SDEEG 33,"[-0.826756, 44.664279]",ECHANGEUR PELOUNEYRES 33380 MIOS,FRS33PMB3315514,MIOS - ECHANGEUR PELOUNEYRES,Voirie,4,Accès libre,24/7,false,FRS33EMB3315514,2024-05-15,2024-05-15T08:30:10.714000+00:00


### Defauts operateur IONITY
Les défauts (220 points de recharge) sont liés à une incohérence entre adresse et coordonnées (une même coordonnée a plusieurs adresses différentes). Ce défaut est lié soit à des erreurs de saisie, soit à des ajouts (ou remplacements) ultérieurs de stations à une même localisation.

Les stations Ionity en erreur ont un id_station_itinerance identique à l'id_pdc_itinerance ainsi qu'un ancien pdc de regroupement par station avec un nom d'enseigne différent (IONITY GMBH), ce qui se traduit par des adresses multiples pour une même coordonnée.

Par exemple, sur l'aire de Mornas [4.732409, 44.194832], on trouve 26 stations avec la même coordonnée, le même identifiant entre point de recharge et station et un nombre de PdC indiqué de 23 pour chaque station.

In [44]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'info@ionity.eu', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
748,42851,info@ionity.eu,IONITY GMBH,"[1.580008, 48.468782]","Aire de Chartres-Gasville, A11 - 28300 Gasvill...",FRIONE126400,IONITY Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1264,2023-03-21,2024-12-06T06:46:06.160000+00:00
749,42856,info@ionity.eu,IONITY GMBH,"[1.416918, 46.313059]","Aire de Boismandé Ouest, A20 - 87160 Saint-Sul...",FRIONE402000,IONITY Boismandé Ouest,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4020,2023-03-21,2024-12-06T06:46:06.160000+00:00
750,42859,info@ionity.eu,IONITY GMBH,"[0.547145, 49.614324]","Aire de services de Bolleville Est, A29, 76210...",FRIONE402300,IONITY Bolleville Est,Station dédiée à la recharge rapide,3,Accès libre,24/7,FALSE,FRIONE4023,2023-03-21,2024-12-06T06:46:06.160000+00:00
751,42863,info@ionity.eu,IONITY GMBH,"[-1.420473, 43.586775]",Aire de Lacq Sud AUDEJOS A64-64170 Lacq,FRIONE402700,IONITY Lacq Audéjos Nord,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4027,2023-03-21,2024-12-06T06:46:06.160000+00:00
752,42869,info@ionity.eu,IONITY GMBH,"[6.623059, 45.196766]","Autoroute de la Maurienne, A43 - Autoport Du F...",FRIONE403700,IONITY Freney,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4037,2023-03-21,2024-12-06T06:46:06.160000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,43725,info@ionity.eu,La Garde,"[3.253033, 44.870544]","Aire de Lagarde, 48200 ALBARET-SAINTE-MARIE",FRIOYE432653,La Garde,Voirie,9,Accès libre,24/7,false,FRIOYE432653,2025-01-02,2025-01-03T00:00:18.731000+00:00
961,43755,info@ionity.eu,Kergoët,"[-2.187398, 48.273996]","Zone d'activités de Kergoët, D46, 22350 Saint-...",FRIOYE436002,Kergoët,Voirie,4,Accès libre,24/7,false,FRIOYE436002,2025-01-02,2025-01-03T00:00:18.731000+00:00
962,43756,info@ionity.eu,Kergoët,"[-2.187398, 48.273996]","Zone d'activités de Kergoët, D46, 22350 Saint-...",FRIOYE436003,Kergoët,Voirie,4,Accès libre,24/7,false,FRIOYE436003,2025-01-02,2025-01-03T00:00:18.731000+00:00
963,43757,info@ionity.eu,Kergoët,"[-2.187398, 48.273996]","Zone d'activités de Kergoët, D46, 22350 Saint-...",FRIOYE436004,Kergoët,Voirie,4,Accès libre,24/7,false,FRIOYE436004,2025-01-02,2025-01-03T00:00:18.731000+00:00


In [45]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[1.580008, 48.468782]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
748,42851,info@ionity.eu,IONITY GMBH,"[1.580008, 48.468782]","Aire de Chartres-Gasville, A11 - 28300 Gasvill...",FRIONE126400,IONITY Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1264,2023-03-21,2024-12-06T06:46:06.160000+00:00
773,42994,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126401,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126401,2025-01-02,2025-01-03T00:00:18.731000+00:00
774,42995,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126402,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126402,2025-01-02,2025-01-03T00:00:18.731000+00:00
775,42996,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126404,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126404,2025-01-02,2025-01-03T00:00:18.731000+00:00
776,42997,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126405,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126405,2025-01-02,2025-01-03T00:00:18.731000+00:00
777,42998,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126451,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126451,2025-01-02,2025-01-03T00:00:18.731000+00:00
778,42999,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126452,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126452,2025-01-02,2025-01-03T00:00:18.731000+00:00
779,43000,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126453,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126453,2025-01-02,2025-01-03T00:00:18.731000+00:00


In [46]:
# example [1.580008, 48.468782]
localisation = itinerance_4.loc[itinerance_4.coordonneesXY == '[1.580008, 48.468782]']
adresses = list(localisation.drop_duplicates('adresse_station')['adresse_station'])
print(adresses)

['Aire de Chartres-Gasville, A11 - 28300 Gasville-Oiséme', 'Chartres-Gasville A11, 28300 Gasville-Oisème']


In [47]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'info@ionity.eu', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
748,42851,info@ionity.eu,IONITY GMBH,"[1.580008, 48.468782]","Aire de Chartres-Gasville, A11 - 28300 Gasvill...",FRIONE126400,IONITY Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1264,2023-03-21,2024-12-06T06:46:06.160000+00:00
749,42856,info@ionity.eu,IONITY GMBH,"[1.416918, 46.313059]","Aire de Boismandé Ouest, A20 - 87160 Saint-Sul...",FRIONE402000,IONITY Boismandé Ouest,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4020,2023-03-21,2024-12-06T06:46:06.160000+00:00
750,42859,info@ionity.eu,IONITY GMBH,"[0.547145, 49.614324]","Aire de services de Bolleville Est, A29, 76210...",FRIONE402300,IONITY Bolleville Est,Station dédiée à la recharge rapide,3,Accès libre,24/7,FALSE,FRIONE4023,2023-03-21,2024-12-06T06:46:06.160000+00:00
751,42863,info@ionity.eu,IONITY GMBH,"[-1.420473, 43.586775]",Aire de Lacq Sud AUDEJOS A64-64170 Lacq,FRIONE402700,IONITY Lacq Audéjos Nord,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4027,2023-03-21,2024-12-06T06:46:06.160000+00:00
752,42869,info@ionity.eu,IONITY GMBH,"[6.623059, 45.196766]","Autoroute de la Maurienne, A43 - Autoport Du F...",FRIONE403700,IONITY Freney,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4037,2023-03-21,2024-12-06T06:46:06.160000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,43725,info@ionity.eu,La Garde,"[3.253033, 44.870544]","Aire de Lagarde, 48200 ALBARET-SAINTE-MARIE",FRIOYE432653,La Garde,Voirie,9,Accès libre,24/7,false,FRIOYE432653,2025-01-02,2025-01-03T00:00:18.731000+00:00
961,43755,info@ionity.eu,Kergoët,"[-2.187398, 48.273996]","Zone d'activités de Kergoët, D46, 22350 Saint-...",FRIOYE436002,Kergoët,Voirie,4,Accès libre,24/7,false,FRIOYE436002,2025-01-02,2025-01-03T00:00:18.731000+00:00
962,43756,info@ionity.eu,Kergoët,"[-2.187398, 48.273996]","Zone d'activités de Kergoët, D46, 22350 Saint-...",FRIOYE436003,Kergoët,Voirie,4,Accès libre,24/7,false,FRIOYE436003,2025-01-02,2025-01-03T00:00:18.731000+00:00
963,43757,info@ionity.eu,Kergoët,"[-2.187398, 48.273996]","Zone d'activités de Kergoët, D46, 22350 Saint-...",FRIOYE436004,Kergoët,Voirie,4,Accès libre,24/7,false,FRIOYE436004,2025-01-02,2025-01-03T00:00:18.731000+00:00


### Defauts operateur BOUYGUES
Les défauts (286 points de recharge) sont liés à une incohérence entre adresse et coordonnées (une même coordonnée a plusieurs adresses différentes).

Les défauts sont majoritairement associés à l'enseigne Ecocharge77 pour laquelle on trouve des stations en doublon entre Bouygues et AlizeCharge. Ce défaut semble être lié à un transfert de bornes d'un opérateur à un autre sans que les anciennes stations aient été supprimées.

Par exemple, pour la coordonnées [3.438183,48.694653], on trouve quatre points de recharge associés à deux stations (une est opérée par Bouygues et l'autre par AlizeCharge).

In [48]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'a.buffler@bouygues-es.com', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2836,94727,a.buffler@bouygues-es.com,Ecocharge77,"[3.438183,48.694653]",route de Brangis,FRS77P77301A,Montceaux-les-Provins,Voirie,2,Accès libre,24/7,false,FRS77EEC7710011,2024-12-24,2024-12-24T09:00:19.626000+00:00
2837,94728,a.buffler@bouygues-es.com,Ecocharge77,"[3.438183,48.694653]",route de Brangis,FRS77P77301A,Montceaux-les-Provins,Voirie,2,Accès libre,24/7,false,FRS77EEC7710012,2024-12-24,2024-12-24T09:00:19.626000+00:00
2838,94729,a.buffler@bouygues-es.com,Ecocharge77,"[2.938246,48.675091]",Avenue du général Leclerc,FRS77P77031A,Bernay-Vilbert - Salle des fêtes,Voirie,2,Accès libre,24/7,false,FRS77EEC771011,2024-12-24,2024-12-24T09:00:19.626000+00:00
2839,94732,a.buffler@bouygues-es.com,Ecocharge77,"[2.938246,48.675091]",Avenue du général Leclerc,FRS77P77031A,Bernay-Vilbert - Salle des fêtes,Voirie,2,Accès libre,24/7,false,FRS77EEC771012,2024-12-24,2024-12-24T09:00:19.626000+00:00
2840,94733,a.buffler@bouygues-es.com,Ecocharge77,"[3.427434,48.851772]",rue Brulée,FRS77P77303A,Montdauphin,Voirie,2,Accès libre,24/7,false,FRS77EEC7710211,2024-12-24,2024-12-24T09:00:19.626000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3157,95212,a.buffler@bouygues-es.com,Ecocharge77,"[2.859685,48.926731]",Place Jean Jaurès,FRS77P77276A,Mareuil-les-Meaux,Voirie,2,Accès libre,24/7,false,FRS77EEC779412,2024-12-24,2024-12-24T09:00:19.626000+00:00
3158,95219,a.buffler@bouygues-es.com,Ecocharge77,"[2.850102,48.349255]",Rue Grande,FRS77P77299A,Morêt-Loing-et-Orvanne - Montarlot,Voirie,2,Accès libre,24/7,false,FRS77EEC779811,2024-12-24,2024-12-24T09:00:19.626000+00:00
3159,95220,a.buffler@bouygues-es.com,Ecocharge77,"[2.850102,48.349255]",Rue Grande,FRS77P77299A,Morêt-Loing-et-Orvanne - Montarlot,Voirie,2,Accès libre,24/7,false,FRS77EEC779812,2024-12-24,2024-12-24T09:00:19.626000+00:00
3160,95221,a.buffler@bouygues-es.com,Ecocharge77,"[2.985625,48.941795]",place de la mairie,FRS77P77300A,Montceaux-les-Meaux,Voirie,2,Accès libre,24/7,false,FRS77EEC779911,2024-12-24,2024-12-24T09:00:19.626000+00:00


In [49]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[2.850102,48.349255]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2708,94545,support@alizecharge.fr,Ecocharge77,"[2.850102,48.349255]","Rue Grande, 77250 MORET-LOING-ORVANNE",FRS77E77316001,MORET-LOING-ORVANNE - Rue Grande,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E773160011,2024-12-31,2024-12-31T11:18:24.916000+00:00
2709,94546,support@alizecharge.fr,Ecocharge77,"[2.850102,48.349255]","Rue Grande, 77250 MORET-LOING-ORVANNE",FRS77E77316001,MORET-LOING-ORVANNE - Rue Grande,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E773160012,2024-12-31,2024-12-31T11:18:24.916000+00:00
3158,95219,a.buffler@bouygues-es.com,Ecocharge77,"[2.850102,48.349255]",Rue Grande,FRS77P77299A,Morêt-Loing-et-Orvanne - Montarlot,Voirie,2,Accès libre,24/7,false,FRS77EEC779811,2024-12-24,2024-12-24T09:00:19.626000+00:00
3159,95220,a.buffler@bouygues-es.com,Ecocharge77,"[2.850102,48.349255]",Rue Grande,FRS77P77299A,Morêt-Loing-et-Orvanne - Montarlot,Voirie,2,Accès libre,24/7,false,FRS77EEC779812,2024-12-24,2024-12-24T09:00:19.626000+00:00


In [50]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[3.438183,48.694653]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2694,94527,support@alizecharge.fr,Ecocharge77,"[3.438183,48.694653]","Route De Brangis, 77151 MONTCEAUX-LES-PROVINS",FRS77E77301001,MONTCEAUX-LES-PROVINS - Route De Brangis,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E773010011,2024-12-31,2024-12-31T11:18:24.916000+00:00
2695,94528,support@alizecharge.fr,Ecocharge77,"[3.438183,48.694653]","Route De Brangis, 77151 MONTCEAUX-LES-PROVINS",FRS77E77301001,MONTCEAUX-LES-PROVINS - Route De Brangis,Voirie,2,Accès libre,Mo-Su 00:00-23:57,False,FRS77E773010012,2024-12-31,2024-12-31T11:18:24.916000+00:00
2836,94727,a.buffler@bouygues-es.com,Ecocharge77,"[3.438183,48.694653]",route de Brangis,FRS77P77301A,Montceaux-les-Provins,Voirie,2,Accès libre,24/7,false,FRS77EEC7710011,2024-12-24,2024-12-24T09:00:19.626000+00:00
2837,94728,a.buffler@bouygues-es.com,Ecocharge77,"[3.438183,48.694653]",route de Brangis,FRS77P77301A,Montceaux-les-Provins,Voirie,2,Accès libre,24/7,false,FRS77EEC7710012,2024-12-24,2024-12-24T09:00:19.626000+00:00


### Cohérence implantation_station - id_station
- 107 pdc sont liés à une erreur de choix d'implantation (couplé à d'autres erreurs)

In [51]:
itinerance_4.loc[~itinerance_4['implantation_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
177,2352,support@alizecharge.fr,ADP Groupe,"[2.577444,49.004299]","Route des Peupliers, 77990 Le Mesnil-Amelot",FRADPE77291004,CDG - Parking PEF - Niveau -2 - Allée 14 & 15,Parking public,32,Accès libre,Mo-Su 00:00-23:57,False,FRADPE772910041,2024-12-31,2024-12-31T11:18:24.916000+00:00
178,2353,support@alizecharge.fr,ADP Groupe,"[2.577444,49.004299]","Route des Peupliers, 77990 Le Mesnil-Amelot",FRADPE77291004,CDG - Parking PEF - Niveau -2 - Allée 14 & 15,Parking public,32,Accès libre,Mo-Su 00:00-23:57,False,FRADPE7729100410,2024-12-31,2024-12-31T11:18:24.916000+00:00
179,2354,support@alizecharge.fr,ADP Groupe,"[2.577444,49.004299]","Route des Peupliers, 77990 Le Mesnil-Amelot",FRADPE77291004,CDG - Parking PEF - Niveau -2 - Allée 14 & 15,Parking public,32,Accès libre,Mo-Su 00:00-23:57,False,FRADPE7729100411,2024-12-31,2024-12-31T11:18:24.916000+00:00
180,2355,support@alizecharge.fr,ADP Groupe,"[2.577444,49.004299]","Route des Peupliers, 77990 Le Mesnil-Amelot",FRADPE77291004,CDG - Parking PEF - Niveau -2 - Allée 14 & 15,Parking public,32,Accès libre,Mo-Su 00:00-23:57,False,FRADPE7729100412,2024-12-31,2024-12-31T11:18:24.916000+00:00
181,2356,support@alizecharge.fr,ADP Groupe,"[2.577444,49.004299]","Route des Peupliers, 77990 Le Mesnil-Amelot",FRADPE77291004,CDG - Parking PEF - Niveau -2 - Allée 14 & 15,Parking public,32,Accès libre,Mo-Su 00:00-23:57,False,FRADPE7729100413,2024-12-31,2024-12-31T11:18:24.916000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3357,100597,customerservice@shellrecharge.com,Shell Fenioux Est,"[-0.60280679, 45.89253519]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE94,Shell Fenioux Est,Voirie,8,Accès réservé,24/7,false,FRSHEE944,2025-01-02,2025-01-03T00:00:18.731000+00:00
3358,100598,customerservice@shellrecharge.com,Shell Fenioux Est,"[-0.60280679, 45.89253519]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE95,Shell Fenioux Est,Station dédiée à la recharge rapide,8,Accès réservé,24/7,false,FRSHEE951,2025-01-02,2025-01-03T00:00:18.731000+00:00
3359,100599,customerservice@shellrecharge.com,Shell Fenioux Est,"[-0.60280679, 45.89253519]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE95,Shell Fenioux Est,Station dédiée à la recharge rapide,8,Accès réservé,24/7,false,FRSHEE952,2025-01-02,2025-01-03T00:00:18.731000+00:00
3360,100600,customerservice@shellrecharge.com,Shell Fenioux Est,"[-0.60280679, 45.89253519]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE95,Shell Fenioux Est,Station dédiée à la recharge rapide,8,Accès réservé,24/7,false,FRSHEE953,2025-01-02,2025-01-03T00:00:18.731000+00:00


### Cohérence nom_station - id_station
- 210 pdc sont associés à une station avec un nom non cohérent

In [52]:
itinerance_4.loc[~itinerance_4['nom_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
18,220,contact@nexteneo.com,Ville d'Annecy,"[6.12983, 45.902285]",[Rue Guillaume Fichet][74000][Annecy],FR0NXPOOLHFX48,Ville d'Annecy - Parking de Bonlieu,Parking public,11,Accès réservé,24/7,FALSE,FR0NXEVSE8BD3L,2024-12-23,2024-12-23T18:18:42.704000+00:00
19,229,contact@nexteneo.com,Ville d'Annecy,"[6.12983, 45.902285]",[Rue Guillaume Fichet][74000][Annecy],FR0NXPOOLHFX48,Ville d'Annecy - Parking de Bonlieu,Parking public,11,Accès réservé,24/7,FALSE,FR0NXEVSE99XBN,2024-12-23,2024-12-23T18:18:42.704000+00:00
20,244,contact@nexteneo.com,Ville d'Annecy,"[6.128716, 45.9]",[1 Esplanade de l'hôtel de ville][74000][Annecy],FR0NXPOOLHFX48,Ville d'Annecy - Hôtel de ville,Parking public,6,Accès réservé,24/7,FALSE,FR0NXEVSEBYTEK,2024-12-23,2024-12-23T18:18:42.704000+00:00
21,250,contact@nexteneo.com,Ville d'Annecy,"[6.128716, 45.9]",[1 Esplanade de l'hôtel de ville][74000][Annecy],FR0NXPOOLHFX48,Ville d'Annecy - Hôtel de ville,Parking public,6,Accès réservé,24/7,FALSE,FR0NXEVSECTDM6,2024-12-23,2024-12-23T18:18:42.704000+00:00
22,252,contact@nexteneo.com,Ville d'Annecy,"[6.12983, 45.902285]",[Rue Guillaume Fichet][74000][Annecy],FR0NXPOOLHFX48,Ville d'Annecy - Parking de Bonlieu,Parking public,11,Accès réservé,24/7,FALSE,FR0NXEVSED4T64,2024-12-23,2024-12-23T18:18:42.704000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4112,115450,support@greenflux.com,ENGIE Vianeo,"[0.048634, 44.293855]",CENTER PARCS LANDES DE GASCOGNE - Parking H - ...,FRVIAP123020,CENTER PARCS LANDES DE GASCOGNE - Parking H,Station dédiée à la recharge rapide,1,Accès libre,24/7,false,FRVIAE20123020022,2024-10-02,2024-10-08T15:23:52.217000+00:00
4113,115451,support@greenflux.com,ENGIE Vianeo,"[0.048634, 44.293855]",CENTER PARCS LANDES DE GASCOGNE - Parking H - ...,FRVIAP123020,CENTER PARCS LANDES DE GASCOGNE - Parking H,Station dédiée à la recharge rapide,1,Accès libre,24/7,false,FRVIAE20123020023,2024-10-02,2024-10-08T15:23:52.217000+00:00
4114,115452,support@greenflux.com,ENGIE Vianeo,"[0.048634, 44.293855]",CENTER PARCS LANDES DE GASCOGNE - Parking H - ...,FRVIAP123020,CENTER PARCS LANDES DE GASCOGNE - Parking H,Station dédiée à la recharge rapide,1,Accès libre,24/7,false,FRVIAE20123020024,2024-10-02,2024-10-08T15:23:52.217000+00:00
4115,115453,support@greenflux.com,ENGIE Vianeo,"[0.048634, 44.293855]",CENTER PARCS LANDES DE GASCOGNE - Parking H - ...,FRVIAP123020,CENTER PARCS LANDES DE GASCOGNE - Parking H,Station dédiée à la recharge rapide,1,Accès libre,24/7,false,FRVIAE20123020025,2024-10-02,2024-10-08T15:23:52.217000+00:00


### Coherence adresse - coordonnees
- 2337 pdc ont une adresse non cohérente avec les coordonnées géographiques

In [53]:
itinerance_4.loc[~itinerance_4['adresse_station - coordonneesXY'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
246,2505,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE91479018,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE914790181,2024-12-31,2024-12-31T11:18:24.916000+00:00
247,2506,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE91479018,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901810,2024-12-31,2024-12-31T11:18:24.916000+00:00
248,2507,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE91479018,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901811,2024-12-31,2024-12-31T11:18:24.916000+00:00
249,2508,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE91479018,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901812,2024-12-31,2024-12-31T11:18:24.916000+00:00
250,2509,support@alizecharge.fr,ADP Groupe,"[2.362554,48.728787]","2 Rue des Transporteurs, 94390 Paray-Vieille-P...",FRADPE91479018,ORY - Parking P3 SILO - Niveau 2,Parking public,16,Accès libre,Mo-Su 00:00-23:57,False,FRADPE9147901813,2024-12-31,2024-12-31T11:18:24.916000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4229,119221,advenir@zeborne.com,GARAGE NELLO CHELLI - DIJON,"[5.065799, 47.354481]",23 RUE DES ARDENNES 21000 Dijon,FRZTLE22AC55087,Toyota - Dijon - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 07:45-12:00,Mo-Fr 13:45-19:00,Sat 09:00-...",false,FRZTLE22AC55087,2021-02-19,2024-12-06T06:45:00.153000+00:00
4230,119232,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2024-12-06T06:44:59.419000+00:00
4231,119233,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2024-12-06T06:44:59.419000+00:00
4232,119234,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2024-12-06T06:44:59.419000+00:00


In [54]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[5.2592028, 46.1746523]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
4230,119232,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2024-12-06T06:44:59.419000+00:00
4231,119233,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2024-12-06T06:44:59.419000+00:00
4232,119234,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2024-12-06T06:44:59.419000+00:00
4233,119235,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57335,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57335,2021-04-08,2024-12-06T06:44:59.419000+00:00


In [55]:
itinerance_4.loc[itinerance_4.adresse_station == 'RN 75 ZI DE NOIREFONTAINE 01250 Montagnat', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
4230,119232,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2024-12-06T06:44:59.419000+00:00
4232,119234,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2024-12-06T06:44:59.419000+00:00


In [56]:
itinerance_4.loc[itinerance_4.adresse_station == ' RN 75 ZI DE NOIREFONTAINE 01250 Montagnat', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
4231,119233,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2024-12-06T06:44:59.419000+00:00
4233,119235,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57335,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57335,2021-04-08,2024-12-06T06:44:59.419000+00:00


### Cohérence station - coordonnées
- 218 pdc sont associés à des stations avec plusieurs coordonnées (majorité AlizeCharge)

In [57]:
itinerance_4.loc[~itinerance_4['coordonneesXY - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
0,180,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E0HKH51125,2022-07-21,2024-12-06T06:46:31.269000+00:00
1,181,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E8OV45243,2022-07-21,2024-12-06T06:46:31.269000+00:00
2,182,acelec@acelec-france.com,Camaïeu France,"[3.207124,50.684984]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EBEQA5957,2022-07-21,2024-12-06T06:46:31.269000+00:00
3,183,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073ECC1C5530,2022-07-21,2024-12-06T06:46:31.269000+00:00
4,184,acelec@acelec-france.com,HOTEL KYRIAD PARIS BEZONS,"[2.212336,48.931356]",80 Av. Gabriel Péri 95870 BEZONS,FR073PKYRIADBEZONS,HOTEL KYRIAD PARIS BEZONS,Parking privé à usage public,1,Accès libre,Mo-Fr 08:00-19:01,false,FR073EJJYD51230,2022-07-21,2024-12-06T06:46:31.269000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4112,115450,support@greenflux.com,ENGIE Vianeo,"[0.048634, 44.293855]",CENTER PARCS LANDES DE GASCOGNE - Parking H - ...,FRVIAP123020,CENTER PARCS LANDES DE GASCOGNE - Parking H,Station dédiée à la recharge rapide,1,Accès libre,24/7,false,FRVIAE20123020022,2024-10-02,2024-10-08T15:23:52.217000+00:00
4113,115451,support@greenflux.com,ENGIE Vianeo,"[0.048634, 44.293855]",CENTER PARCS LANDES DE GASCOGNE - Parking H - ...,FRVIAP123020,CENTER PARCS LANDES DE GASCOGNE - Parking H,Station dédiée à la recharge rapide,1,Accès libre,24/7,false,FRVIAE20123020023,2024-10-02,2024-10-08T15:23:52.217000+00:00
4114,115452,support@greenflux.com,ENGIE Vianeo,"[0.048634, 44.293855]",CENTER PARCS LANDES DE GASCOGNE - Parking H - ...,FRVIAP123020,CENTER PARCS LANDES DE GASCOGNE - Parking H,Station dédiée à la recharge rapide,1,Accès libre,24/7,false,FRVIAE20123020024,2024-10-02,2024-10-08T15:23:52.217000+00:00
4115,115453,support@greenflux.com,ENGIE Vianeo,"[0.048634, 44.293855]",CENTER PARCS LANDES DE GASCOGNE - Parking H - ...,FRVIAP123020,CENTER PARCS LANDES DE GASCOGNE - Parking H,Station dédiée à la recharge rapide,1,Accès libre,24/7,false,FRVIAE20123020025,2024-10-02,2024-10-08T15:23:52.217000+00:00


In [58]:
itinerance_5.loc[itinerance_5.id_station_itinerance	 == 'FRTLSE31555040', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
3486,108377,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666,43.6039735]","61 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE - Bd Lazare Carnot,Voirie,1,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550401,2024-12-31,2024-12-31T11:18:24.916000+00:00
3487,108378,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666,43.6039735]","61 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE - Bd Lazare Carnot,Voirie,1,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550402,2024-12-31,2024-12-31T11:18:24.916000+00:00
3488,108379,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026,43.6040701]","63 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550403,2024-12-31,2024-12-31T11:18:24.916000+00:00
3489,108380,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026,43.6040701]","63 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550404,2024-12-31,2024-12-31T11:18:24.916000+00:00
3490,108381,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026,43.6040701]","63 Bd Lazare Carnot, 31000 TOULOUSE",FRTLSE31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:57,False,FRTLSE315550405,2024-12-31,2024-12-31T11:18:24.916000+00:00


In [59]:
itinerance_5.loc[itinerance_5.id_station_itinerance	 == 'FRFR1PEHNKRRPQNR', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


- exemple : une station avec plusieurs coordonnées -> incompréhension distinction station / pdc

In [60]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR073PCAMAIEUFR', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
0,180,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E0HKH51125,2022-07-21,2024-12-06T06:46:31.269000+00:00
1,181,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E8OV45243,2022-07-21,2024-12-06T06:46:31.269000+00:00
2,182,acelec@acelec-france.com,Camaïeu France,"[3.207124,50.684984]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EBEQA5957,2022-07-21,2024-12-06T06:46:31.269000+00:00
3,183,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073ECC1C5530,2022-07-21,2024-12-06T06:46:31.269000+00:00
5,185,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EK0BJ522,2022-07-21,2024-12-06T06:46:31.269000+00:00
6,186,acelec@acelec-france.com,Camaïeu France,"[3.207047,50.685013]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EMZLV5283,2022-07-21,2024-12-06T06:46:31.269000+00:00
7,187,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EO0AV51091,2022-07-21,2024-12-06T06:46:31.269000+00:00
8,188,acelec@acelec-france.com,Camaïeu France,"[3.207187,50.684964]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP7QU5233,2022-07-21,2024-12-06T06:46:31.269000+00:00
9,189,acelec@acelec-france.com,Camaïeu France,"[3.206962,50.685049]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP9LD51287,2022-07-21,2024-12-06T06:46:31.269000+00:00
10,190,acelec@acelec-france.com,Camaïeu France,"[3.207433,50.684876]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EQ1GI51176,2022-07-21,2024-12-06T06:46:31.269000+00:00


- exemple : une station avec plusieurs coordonnées -> regroupement de stations ?

In [61]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR55CPBP514', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


- exemple : plusieurs stations d'opérateurs différents avec les mêmes coordonnées -> suppression non effectuée lors d'un changement d'opérateur ?

In [62]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[0.08745032, 48.40887156]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


In [63]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-4.374469, 48.471878]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


- exemple : quatre stations avec une localisation identique mais deux adresses différentes + id identique entre station et pdc -> erreur d'adresse et erreur d'id_station ?

In [64]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[-0.366184, 43.332508]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### Incoherence nbre_pdc
- 583 pdc ont un champ 'nbre_pdc' mal documenté

In [65]:
itinerance_4.loc[~itinerance_4['nbre_pdc - id_station_itinerance'], relations][200:215]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
557,26550,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[3.1572541980319087, 45.84120888197422]",84 Chemin de Donnavignat 63360 Gerzat,FRETIP63164A,e-Totem - Fasthôtel Gerzat,Parking privé à usage public,5,Accès libre,24/7,false,FRETIE63164A15,2024-12-23,2024-12-23T13:38:51.439000+00:00
558,26551,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[3.1572541980319087, 45.84120888197422]",84 Chemin de Donnavignat 63360 Gerzat,FRETIP63164A,e-Totem - Fasthôtel Gerzat,Parking privé à usage public,2,Accès libre,24/7,false,FRETIE63164A21,2024-12-23,2024-12-23T13:38:51.439000+00:00
559,26552,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[3.1572541980319087, 45.84120888197422]",84 Chemin de Donnavignat 63360 Gerzat,FRETIP63164A,e-Totem - Fasthôtel Gerzat,Parking privé à usage public,2,Accès libre,24/7,false,FRETIE63164A22,2024-12-23,2024-12-23T13:38:51.439000+00:00
560,26577,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[-1.5385403559826865, 43.46612969154638]",Parking Iraty 64200 Biarritz,FRETIP64122D,e-Totem - BIARRITZ Parking Iraty,Parking privé à usage public,7,Accès libre,24/7,false,FRETIE64122D11,2024-12-23,2024-12-23T13:38:51.439000+00:00
561,26578,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[-1.5385403559826865, 43.46612969154638]",Parking Iraty 64200 Biarritz,FRETIP64122D,e-Totem - BIARRITZ Parking Iraty,Parking privé à usage public,7,Accès libre,24/7,false,FRETIE64122D12,2024-12-23,2024-12-23T13:38:51.439000+00:00
562,26579,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[-1.5385403559826865, 43.46612969154638]",Parking Iraty 64200 Biarritz,FRETIP64122D,e-Totem - BIARRITZ Parking Iraty,Parking privé à usage public,7,Accès libre,24/7,false,FRETIE64122D13,2024-12-23,2024-12-23T13:38:51.439000+00:00
563,26580,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[-1.5385403559826865, 43.46612969154638]",Parking Iraty 64200 Biarritz,FRETIP64122D,e-Totem - BIARRITZ Parking Iraty,Parking privé à usage public,7,Accès libre,24/7,false,FRETIE64122D14,2024-12-23,2024-12-23T13:38:51.439000+00:00
564,26581,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[-1.5385403559826865, 43.46612969154638]",Parking Iraty 64200 Biarritz,FRETIP64122D,e-Totem - BIARRITZ Parking Iraty,Parking privé à usage public,7,Accès libre,24/7,false,FRETIE64122D15,2024-12-23,2024-12-23T13:38:51.439000+00:00
565,26582,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[-1.5385403559826865, 43.46612969154638]",Parking Iraty 64200 Biarritz,FRETIP64122D,e-Totem - BIARRITZ Parking Iraty,Parking privé à usage public,7,Accès libre,24/7,false,FRETIE64122D16,2024-12-23,2024-12-23T13:38:51.439000+00:00
566,26583,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[-1.5385403559826865, 43.46612969154638]",Parking Iraty 64200 Biarritz,FRETIP64122D,e-Totem - BIARRITZ Parking Iraty,Parking privé à usage public,7,Accès libre,24/7,false,FRETIE64122D17,2024-12-23,2024-12-23T13:38:51.439000+00:00


### coherence station - enseigne
- 5 stations ont plusieurs noms d'enseigne

In [66]:
itinerance_4.loc[~itinerance_4['nom_enseigne - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
611,30463,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS1,2022-12-09,2024-12-06T06:49:25.999000+00:00
612,30464,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS2,2022-12-09,2024-12-06T06:49:25.999000+00:00
613,30465,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS3,2022-12-09,2024-12-06T06:49:25.999000+00:00
1099,53133,support@eoliberty.fr,ITM CLERMONT DE L'OISE 1,"[2.4002659, 49.3819747]",Rue Gérard de Nerval Clermont de l'oise,FRLIBP541399,ITM CLERMONT DE L'OISE 1,Voirie,2,Accès libre,24/7,FALSE,FRLIBE3794441,2024-10-07,2024-12-30T15:52:57.013000+00:00
1100,53134,support@eoliberty.fr,ITM CLERMONT DE L'OISE 1,"[2.4002659, 49.3819747]",Rue Gérard de Nerval Clermont de l'oise,FRLIBP541399,ITM CLERMONT DE L'OISE 1,Voirie,2,Accès libre,24/7,FALSE,FRLIBE3794443,2024-10-07,2024-12-30T15:52:57.013000+00:00
1101,53135,support@eoliberty.fr,ITM CLERMONT DE L'OISE 2,"[2.4002659, 49.3819747]",Rue Gérard de Nerval Clermont de l'oise,FRLIBP541399,ITM CLERMONT DE L'OISE 2,Voirie,2,Accès libre,24/7,FALSE,FRLIBE3794444,2024-10-07,2024-12-30T15:52:57.013000+00:00
1102,53136,support@eoliberty.fr,ITM CLERMONT DE L'OISE 2,"[2.4002659, 49.3819747]",Rue Gérard de Nerval Clermont de l'oise,FRLIBP541399,ITM CLERMONT DE L'OISE 2,Voirie,2,Accès libre,24/7,FALSE,FRLIBE3794445,2024-10-07,2024-12-30T15:52:57.013000+00:00
1142,56001,contact@rirodo.fr,LAST MILES SOLUTIONS,"[5.39, 43.27]",20 Rue Negresko,FRRIRE2804856,PARKING NEGRESKO,Parking privé à usage public,2,Accès libre,24/7,false,FRLMSE1897228,2023-09-06,2024-12-06T06:47:15.989000+00:00
1451,60254,support@mobilize-ps.fr,JRA SAUMUR,"[-0.059981, 47.274681]","\tBd Vallerots, 49400 Saumur",FRMBZEEVHJW,JRA SAUMUR,Parking privé à usage public,1,Accès libre,"Mo-Fr 07:30-12:00,Mo-Fr 14:00-19:00",False,FRMBZEEVHJW1,2024-07-10,2024-11-15T16:46:22.342000+00:00
1452,60255,support@mobilize-ps.fr,JRA Saumur,"[-0.05996536452405448, 47.27462246981463]","Bd Vallerots, 49400 Saumur",FRMBZEEVHJW,JRA Saumur,Parking privé à usage public,1,Accès libre,24/7,False,FRMBZEEVHJW2,2024-07-10,2024-11-15T16:46:22.342000+00:00


In [67]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRFR1EAYRU', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
29517,29537,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel jean colonna d'ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU1,2022-12-25,2024-12-06T06:49:41.213000+00:00
29519,29539,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU2,2022-08-25,2024-12-06T06:49:41.213000+00:00
29521,29541,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU3,2022-08-25,2024-12-06T06:49:41.213000+00:00
30443,30463,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS1,2022-12-09,2024-12-06T06:49:25.999000+00:00
30444,30464,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS2,2022-12-09,2024-12-06T06:49:25.999000+00:00
30445,30465,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS3,2022-12-09,2024-12-06T06:49:25.999000+00:00


In [68]:
itinerance_init.loc[itinerance_init.nom_enseigne == 'RME Loos (FR-FR1)', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
29517,29537,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel jean colonna d'ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU1,2022-12-25,2024-12-06T06:49:41.213000+00:00
29519,29539,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU2,2022-08-25,2024-12-06T06:49:41.213000+00:00
30443,30463,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS1,2022-12-09,2024-12-06T06:49:25.999000+00:00
30444,30464,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS2,2022-12-09,2024-12-06T06:49:25.999000+00:00
33846,33866,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.009613, 50.617509]",12 Rue Salengro LOOS,FRFR1EPJXS,"Loos, Parking CIL",Parking public,3,Accès libre,24/7,FALSE,FRFR1EPJXS1,2022-10-04,2024-12-06T06:49:29.527000+00:00
33848,33868,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.009613, 50.617509]",12 Rue Salengro LOOS,FRFR1EPJXS,"Loos, Parking CIL",Parking public,3,Accès libre,24/7,FALSE,FRFR1EPJXS2,2022-10-04,2024-12-06T06:49:29.527000+00:00


### cohérence station - horaires
- exemple de station avec plusieurs types d'horaire (15)

In [69]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS23D2302001', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### cohérence station - deux-roues
- exemple de station avec plusieurs indications d'accessibilité deux-roues (583)

In [70]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRV75PPX1716', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
111480,112497,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,TRUE,FRV75EPX17161,2023-07-06,2024-10-17T12:07:34.504000+00:00
111481,112498,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,TRUE,FRV75EPX17162,2023-07-06,2024-10-17T12:07:34.504000+00:00
111482,112499,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17163,2023-07-06,2024-10-17T12:07:34.504000+00:00
111483,112500,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17164,2023-07-06,2024-10-17T12:07:34.504000+00:00
111484,112501,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17165,2023-07-06,2024-10-17T12:07:34.504000+00:00
111485,112502,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17166,2023-07-06,2024-10-17T12:07:34.504000+00:00
111486,112503,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17167,2023-07-06,2024-10-17T12:07:34.504000+00:00


### stations avec pdc > 350 kW

- exemple de station avec une puissance de plus de 350 kW

In [85]:
data['p_nom'] = data['puissance_nominale'].str.replace(",", ".")

In [89]:
data['p_nom'] = data['p_nom'].astype('float')

In [101]:
data350 = copy(data.loc[data["p_nom"] > 350, :])

In [102]:
data350['index'] = 0

In [103]:
data350.loc[:, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
20,0,roaming@zunder.com,Zunder,"[2.240692 , 47.095986]","A71/E11, km200, Aire de Bourges-Marmagne, Mar...",ESZUNP2454085589633028277,Zunder/122319,Station dédiée à la recharge rapide,48,Accès libre,24/7,False,ESZUNEFR1801ER01,2024-12-27,2025-01-01T03:01:18+00:00
21,0,roaming@zunder.com,Zunder,"[2.240692 , 47.095986]","A71/E11, km200, Aire de Bourges-Marmagne, Mar...",ESZUNP2454085589633028277,Zunder/122319,Station dédiée à la recharge rapide,48,Accès libre,24/7,False,ESZUNEFR1801ER02,2024-12-27,2025-01-01T03:01:18+00:00
22,0,roaming@zunder.com,Zunder,"[2.240692 , 47.095986]","A71/E11, km200, Aire de Bourges-Marmagne, Mar...",ESZUNP2454085589633028277,Zunder/122319,Station dédiée à la recharge rapide,48,Accès libre,24/7,False,ESZUNEFR1801ER03,2024-12-27,2025-01-01T03:01:18+00:00
23,0,roaming@zunder.com,Zunder,"[2.240692 , 47.095986]","A71/E11, km200, Aire de Bourges-Marmagne, Mar...",ESZUNP2454085589633028277,Zunder/122319,Station dédiée à la recharge rapide,48,Accès libre,24/7,False,ESZUNEFR1801ER04,2024-12-27,2025-01-01T03:01:18+00:00
24,0,roaming@zunder.com,Zunder,"[2.240692 , 47.095986]","A71/E11, km200, Aire de Bourges-Marmagne, Mar...",ESZUNP2454085589633028277,Zunder/122319,Station dédiée à la recharge rapide,48,Accès libre,24/7,False,ESZUNEFR1801ER05,2024-12-27,2025-01-01T03:01:18+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115063,0,support@greenflux.com,ENGIE Vianeo,"[4.884340, 44.900180]",B&B HOTEL VALENCE SUD - Avenue des Auréats 51 ...,FRVIAP122072,ENGIE Vianeo - B&B HOTEL VALENCE SUD,Station dédiée à la recharge rapide,1,Accès libre,24/7,false,FRVIAE20122072022,2024-10-02,2024-10-08T15:23:52.217000+00:00
115064,0,support@greenflux.com,ENGIE Vianeo,"[4.884340, 44.900180]",B&B HOTEL VALENCE SUD - Avenue des Auréats 51 ...,FRVIAP122072,ENGIE Vianeo - B&B HOTEL VALENCE SUD,Station dédiée à la recharge rapide,1,Accès libre,24/7,false,FRVIAE20122072031,2024-10-02,2024-10-08T15:23:52.217000+00:00
115065,0,support@greenflux.com,ENGIE Vianeo,"[4.884340, 44.900180]",B&B HOTEL VALENCE SUD - Avenue des Auréats 51 ...,FRVIAP122072,ENGIE Vianeo - B&B HOTEL VALENCE SUD,Station dédiée à la recharge rapide,1,Accès libre,24/7,false,FRVIAE20122072032,2024-10-02,2024-10-08T15:23:52.217000+00:00
115066,0,support@greenflux.com,ENGIE Vianeo,"[4.884340, 44.900180]",B&B HOTEL VALENCE SUD - Avenue des Auréats 51 ...,FRVIAP122072,ENGIE Vianeo - B&B HOTEL VALENCE SUD,Station dédiée à la recharge rapide,1,Accès libre,24/7,false,FRVIAE20122072041,2024-10-02,2024-10-08T15:23:52.217000+00:00


In [104]:
data350['contact_operateur'].unique()

array(['roaming@zunder.com', 'info.allego.france@allego.eu',
       'bppulsefranceops@bp.com', 'info@yaway-recharge.eu',
       'exploitation@yaway-recharge.eu', 'roaming-dev@road.io',
       'support@fastned.nl', 'bornes@lidl.fr', 'irve@soregies.fr',
       'ops@obornes.fr', 'contact@sowattsolutions.com',
       'support@alizecharge.fr', 'support@greenflux.com'], dtype=object)